<a href="https://colab.research.google.com/github/alexisalduncin/Decision-Trees/blob/main/Opp_Assesment_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
!pip install xlsxwriter
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import re
from openpyxl import load_workbook
from datetime import datetime

from collections import defaultdict
pd.set_option('display.max_columns', None)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


JSON CATALOG for excel column mapping

In [ ]:
def load_or_create_catalog(catalog_file='column_name_catalog.json'):
    if os.path.exists(catalog_file):
        with open(catalog_file, 'r') as f:
            return json.load(f)
    else:
        return {
            'Supplier ID': ['Supplier ID', 'Supplier Number', 'Vendor ID', 'Vendor Number', 'Supplier', 'Vendor'],
            'Supplier Name': ['Supplier Name', 'Vendor Name', 'Supplier', 'Vendor'],
            'Item Description': ['Item Description', 'Product Description', 'Description', 'Item Name', 'Product Name'],
            'SKU': ['SKU', 'Item Number', 'Product ID', 'Product Code', 'Item Code', 'Part Number'],
            'Item Group': ['Item Group', 'Item Category', 'Product Group', 'Product Category'],
            'Product Group': ['Product Group', 'Product Category', 'Item Group', 'Item Category'],
            'UOM': ['UOM', 'Unit of Measure', 'Purchase UOM', 'Order UOM'],
            'Quantity': ['Quantity', 'Order Qty', 'Ordered Quantity', 'Qty'],
            'Quantity in UOM': ['Qty Received In Purch UOM', 'Order Qty UOM', 'Quantity in Purchase UOM'],
            'Price': ['Price', 'Purchase Price', 'Unit Price', 'Cost'],
            'Date': ['Received Date', 'Purchase Date', 'Order Date', 'Transaction Date', 'Invoice Date']
        }
def save_catalog(catalog, catalog_file='column_name_catalog.json'):
    """
    Save the catalog to a JSON file.

    Args:
    catalog (dict): The catalog dictionary to be saved.
    catalog_file (str): The filename where the catalog will be saved.
    """
    with open(catalog_file, 'w') as f:
        json.dump(catalog, f, indent=4)

def find_matching_column(df_columns, catalog):
    column_mapping = {}
    unmatched_columns = []
    df_columns = pd.Index(df_columns)

    for standard_name, variations in catalog.items():
        match_found = False
        for possible_name in variations:
            exact_match = df_columns[df_columns.str.lower() == possible_name.lower()]
            if len(exact_match) > 0:
                column_mapping[standard_name] = exact_match[0]
                match_found = True
                break

            partial_matches = [col for col in df_columns if possible_name.lower() in col.lower()]
            if partial_matches:
                column_mapping[standard_name] = partial_matches[0]
                match_found = True
                break

        if not match_found:
            unmatched_columns.append(standard_name)

    return column_mapping, unmatched_columns
def update_catalog(catalog, new_mappings):
    """
    Update the existing catalog with new column name mappings.

    Args:
    catalog (dict): The existing catalog dictionary.
    new_mappings (dict): A dictionary of new mappings to add or update.

    Returns:
    dict: The updated catalog dictionary.
    """
    for standard_name, variations in new_mappings.items():
        if standard_name in catalog:
            catalog[standard_name].extend([v for v in variations if v not in catalog[standard_name]])
        else:
            catalog[standard_name] = variations
    return catalog

def interactive_column_mapping(df_columns, catalog):
    column_mapping, unmatched_columns = find_matching_column(df_columns, catalog)

    print("Initial column mapping:")
    for standard_name, actual_name in column_mapping.items():
        print(f"{standard_name}: {actual_name}")

    print("\nUnmatched columns:")
    for col in unmatched_columns:
        print(f"No match found for '{col}'")

    new_mappings = {}
    for col in unmatched_columns:
        user_input = input(f"\nEnter the correct column name for '{col}' (or press Enter to skip): ").strip()
        if user_input:
            if user_input in column_mapping.values():
                print(f"Error: '{user_input}' is already mapped to another column. Please choose a different name.")
            else:
                column_mapping[col] = user_input
                new_mappings[col] = [user_input]

    if new_mappings:
        catalog = update_catalog(catalog, new_mappings)
        save_catalog(catalog)
        print("\nCatalog updated with new mappings.")

    return column_mapping, catalog

def process_file(df, column_mapping):
    # Create a reverse mapping (from original column names to standard names)
    reverse_mapping = {v: k for k, v in column_mapping.items()}

    # Rename columns based on the mapping
    df_renamed = df.rename(columns=reverse_mapping)

    # Keep only the columns that were successfully mapped
    mapped_columns = [col for col in df_renamed.columns if col in column_mapping.keys()]
    df_renamed = df_renamed[mapped_columns]

    # Perform some basic processing
    if 'Date' in df_renamed.columns:
        df_renamed['Date'] = pd.to_datetime(df_renamed['Date'], errors='coerce')
    if 'Price' in df_renamed.columns:
        df_renamed['Price'] = pd.to_numeric(df_renamed['Price'], errors='coerce')
    if 'Quantity' in df_renamed.columns:
        df_renamed['Quantity'] = pd.to_numeric(df_renamed['Quantity'], errors='coerce')

    return df_renamed

def etl_process(df):
    catalog = load_or_create_catalog()

    print("Original DataFrame columns:")
    print(df.columns.tolist())

    column_mapping, updated_catalog = interactive_column_mapping(df.columns, catalog)

    processed_df = process_file(df, column_mapping)

    print("\nProcessed DataFrame:")
    print(processed_df.head())
    print("\nProcessed DataFrame columns:")
    print(processed_df.columns.tolist())

    return processed_df, updated_catalog

# Example usage
def create_sample_dataframe():
    data = {
        'Vendor ID': ['V001', 'V002', 'V003', 'V004', 'V005'],
        'Product Name': ['Widget A', 'Gadget B', 'Tool C', 'Device D', 'Item E'],
        'Purchase Date 2023': ['2023-01-15', '2023-02-20', '2023-03-25', '2023-04-30', '2023-05-05'],
        'Unit Cost': [100.50, 200.75, 150.25, 300.00, 175.50],
        '#SKU': ['SKU001', 'SKU002', 'SKU003', 'SKU004', 'SKU005'],
        'Amount': ['1', '2', '3', '4', '5'],
        'Itm grp': ['Electronics', 'Tools', 'Hardware', 'Accessories', 'Parts']
    }
    return pd.DataFrame(data)

def run_example():
    df = create_sample_dataframe()
    processed_df, updated_catalog = etl_process(df)
    print("\nETL process completed. DataFrame shape:", processed_df.shape)
    print("\nUpdated Catalog:")
    print(json.dumps(updated_catalog, indent=2))

if __name__ == "__main__":
    run_example()

Original DataFrame columns:
['Vendor ID', 'Product Name', 'Purchase Date 2023', 'Unit Cost', '#SKU', 'Amount', 'Itm grp']
Initial column mapping:
Supplier ID: Vendor ID
Supplier Name: Vendor ID
Item Description: Product Name
SKU: #SKU
Price: Unit Cost
Date: Purchase Date 2023

Unmatched columns:
No match found for 'Item Group'
No match found for 'Product Group'
No match found for 'UOM'
No match found for 'Quantity'
No match found for 'Quantity in UOM'

Enter the correct column name for 'Item Group' (or press Enter to skip): 

Enter the correct column name for 'Product Group' (or press Enter to skip): 

Enter the correct column name for 'UOM' (or press Enter to skip): 

Enter the correct column name for 'Quantity' (or press Enter to skip): 

Enter the correct column name for 'Quantity in UOM' (or press Enter to skip): 

Processed DataFrame:
  Supplier Name Item Description       Date   Price     SKU
0          V001         Widget A 2023-01-15  100.50  SKU001
1          V002         Gadg

### Client uploads file

In [ ]:
def process_client_input(file_path):
    """
    Process the client's input Excel file.

    Args:
    file_path (str): The path to the client's Excel file.

    Returns:
    tuple: A tuple containing:
        - pd.DataFrame: The processed DataFrame with mapped columns and additional required columns.
        - dict: The updated catalog.
    """
    # Load the catalog with the function
    catalog = load_or_create_catalog()

    # Extract client name from the file name to put on table
    client_name = os.path.basename(file_path).split(' - ')[0]

    # Load the workbook
    wb = load_workbook(file_path, read_only=True, data_only=True)

    # Find the sheet with 'Data' in its name
    data_sheet = next((sheet for sheet in wb.sheetnames if 'data' in sheet.lower()), None)
    if not data_sheet:
        raise ValueError("No sheet with 'Data' in its name found in the workbook.")

    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=data_sheet)

    print("Original DataFrame columns:")
    print(df.columns.tolist())

    # Perform interactive column mapping
    column_mapping, updated_catalog = interactive_column_mapping(df.columns, catalog)

    # Process the file with the new mapping
    df = process_file(df, column_mapping)

    # Filter the DataFrame to include only the columns from our catalog
    catalog_columns = list(column_mapping.keys())
    df = df[catalog_columns]

    # Add required columns
    if 'Quantity in UOM' in df.columns and 'Price' in df.columns:
        df['Total'] = df['Quantity in UOM'] * df['Price']
    else:
        print("Warning: 'Quantity in UOM' or 'Price' column not found. 'Total' column not added.")

    df['Stage'] = 'Opportunity Assessment'
    df['Client Name'] = client_name

    print("\nProcessed DataFrame:")
    print(df.head())
    print("\nProcessed DataFrame columns:")
    print(df.columns.tolist())

    return df, updated_catalog

def run_example():
    """
    Run an example of the client input processing.

    This function processes a sample Excel file and displays the results.

    Returns:
    pd.DataFrame: The processed DataFrame.
    """
    # File path (replace with Azure Blob Storage path when moving to Azure)
    file_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Transcendia - Opportunity Assessment.xlsx"

    # Process the client input
    processed_df, updated_catalog = process_client_input(file_path)

    # Here you would typically save the processed DataFrame or perform further analysis
    print("\nProcessing completed. DataFrame shape:", processed_df.shape)
    print("\nFirst few rows of the processed DataFrame:")
    print(processed_df.head())

    # Display the updated catalog
    print("\nUpdated Catalog:")
    print(json.dumps(updated_catalog, indent=2))

    return processed_df

# Run the example and store the result in a global variable
df = run_example()

if __name__ == "__main__":
    pass

Original DataFrame columns:
['Supplier Name', 'Supplier Number', 'Supplier Group Name', 'Supplier Group Number', 'Product', 'Product Description', 'Gauge', 'Item Group', 'Product Group', 'Color', 'Finish', 'Material Code', 'Material Description', 'Width', 'Sheet Length', 'Division', 'Received Year', 'Purchase Order Date', 'Facility Code', 'Warehouse', 'Pounds Received', 'Purchase Price', 'Order Qty', 'Purchase UOM', 'Qty Received In Purch UOM', 'Cost', 'Purchase Order Number', 'Purchase Order Line Number', 'Purchase Order Type', 'Purchase Order Our Reference Number', 'PO Line Order Type', 'PO Line Our Reference Number', 'Received Date', 'System', 'Supplier Item Number', 'BFMPC', 'Requested Date', 'Confirmed Date', 'Final Destination', "Price * Qty Rec'd", 'Include?', 'Include for Savings?', 'Parent Vendor', 'ST Category', 'ST Sub-Category', 'Opportunity Term', 'Est. Savings % (Low)', 'Est. Savings % (High)', 'Est Savings $ (Low)', 'Est. Savings $ (High)', 'Market Place Designation']
In

Azure environement example

In [ ]:
"""
# In Azure environment:
from azure.storage.blob import BlobServiceClient

def read_excel_from_blob(container_name, blob_name):
    connection_string = "your_connection_string"
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)

    with BytesIO(blob_client.download_blob().readall()) as input_file:
        return pd.read_excel(input_file)

# Then use this function instead of pd.read_excel in process_client_input
"""

'\n# In Azure environment:\nfrom azure.storage.blob import BlobServiceClient\n\ndef read_excel_from_blob(container_name, blob_name):\n    connection_string = "your_connection_string"\n    blob_service_client = BlobServiceClient.from_connection_string(connection_string)\n    container_client = blob_service_client.get_container_client(container_name)\n    blob_client = container_client.get_blob_client(blob_name)\n\n    with BytesIO(blob_client.download_blob().readall()) as input_file:\n        return pd.read_excel(input_file)\n\n# Then use this function instead of pd.read_excel in process_client_input\n'

Show initial DataFrame

In [ ]:
df.head()

,Supplier ID,Supplier Name,Item Description,SKU,Item Group,Product Group,UOM,Quantity,Quantity in UOM,Price,Date,Total,Stage,Client Name
0,55311,Mid-Ohio Wood Products,"PALLET 42X42""",,PALLET,PACKAGING MATERIAL,EA,200.0,100.0,18.7500,2024-06-26,1875.00,Opportunity Assessment,Transcendia
1,9141,Bunzl Processor Division,OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,12.0,12.0,4.9900,2024-07-03,59.88,Opportunity Assessment,Transcendia
2,89437,"Hel, Inc.",SERVICES-NON MATERIAL,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.0,1.0,1637.0000,2024-03-17,1637.00,Opportunity Assessment,Transcendia
3,54448,"Mactac, Inc.",.00400 RM FPVC.TC2N.PS CL 0,4-1550,TRANSCLING II,Vinyl Print family,FT,10000.0,10000.0,0.8165,2024-07-30,8165.00,Opportunity Assessment,Transcendia
4,89959,"Rocheux International, Inc.",.01500 RM PVC WHTOPQ MM,BPGBMXX,VINYL GROUP,Vinyl Print family,LBS,15000.0,6489.0,1.7600,2024-01-05,11420.64,Opportunity Assessment,Transcendia


generate_parent_ID(name, counter):

*   Creates a unique Parent Vendor ID from a supplier's name
*   Uses the first letters of up to three words in the name
*   Adds a number to ensure uniqueness
*   For Supplier ID it assigns the IDs using a counter for each Parent Vendor


load_unified_vendor_lookup(path):

*   Loads an existing vendor lookup table from an Excel file


update_vendor_lookup(unified_lookup, new_suppliers):

*   Updates the vendor lookup table with new suppliers
*   Initializes counters for Parent Vendor IDs and Vendor IDs based on existing entries
*   For each new supplier:

*   Generates a new Parent Vendor ID
*   Creates a new Vendor ID
*   Adds a new entry with default values (empty strings and 0s)for categories and savings estimates


*   Combines new entries with the existing lookup table
*   Maintains existing IDs and generates new ones for new suppliers
*   Ensures each supplier has a unique identifier
*   Allows for tracking of parent-child relationships between vendors


In [ ]:
def generate_parent_ID(name, counter):
    """Generate a Parent Vendor ID from a name."""
    base = ''.join(word[0] for word in str(name).upper().split()[:3])
    counter[base] += 1
    return f"{base}{counter[base]:03d}"

def load_unified_vendor_lookup(path):
    """Load the unified Vendor Look Up table from the given path."""
    return pd.read_excel(path)

def update_vendor_lookup(unified_lookup, new_suppliers):
    """Update the unified Vendor Look Up table with new suppliers."""
    parent_ID_counter = defaultdict(int)
    vendor_ID_counter = defaultdict(lambda: defaultdict(int))

    # Initialize counters based on existing IDs
    for _, row in unified_lookup.iterrows():
        parent_ID = row['Parent Vendor ID']
        if pd.notna(parent_ID):
            base = parent_ID[:3]
            num = int(parent_ID[3:])
            parent_ID_counter[base] = max(parent_ID_counter[base], num)

            vendor_ID = row['Vendor ID']
            if pd.notna(vendor_ID):
                vendor_num = int(vendor_ID.split('-')[1])
                vendor_ID_counter[parent_ID][row['Parent Vendor']] = max(
                    vendor_ID_counter[parent_ID][row['Parent Vendor']], vendor_num)

    # Process new suppliers
    new_entries = []
    for supplier in new_suppliers:
        if supplier not in unified_lookup['Supplier Name'].values:
            parent_ID = generate_parent_ID(supplier, parent_ID_counter)
            vendor_ID_counter[parent_ID][supplier] += 1
            vendor_ID = f"{parent_ID}-{vendor_ID_counter[parent_ID][supplier]}"

            new_entry = {
                'Supplier Name': supplier,
                'Parent Vendor': supplier,
                'Parent Vendor ID': parent_ID,
                'Vendor ID': vendor_ID,
                'ST Category': '',
                'ST Sub-Category': '',
                'Opportunity Term': '',
                'Est. Savings % (Low)': 0,
                'Est. Savings % (High)': 0,
                'Marketplace Designation': '',
                'Notes': ''
            }
            new_entries.append(new_entry)

    # Concatenate new entries with existing table
    if new_entries:
        new_entries_df = pd.DataFrame(new_entries)
        updated_lookup = pd.concat([unified_lookup, new_entries_df], ignore_index=True)
    else:
        updated_lookup = unified_lookup

    return updated_lookup

Merging Data:

*  Combines the original DataFrame (df) with the unified vendor lookup table
*  Uses a left join on the 'Supplier Name' column


Calculating Total Cost:

*  Checks if 'Total' column already exists
*  If not, attempts to calculate it by multiplying 'Quantity in UOM' and 'Price'
*  If unable to calculate (due to missing columns), sets 'Total' to NaN and issues a warning


Estimating Savings:

*  Calculates low-end estimated savings: 'Est. Savings (Low)'

*  Multiplies 'Total' by 'Est. Savings % (Low)' (converted to decimal)


*  Calculates high-end estimated savings: 'Est. Savings (High)'

*  Multiplies 'Total' by 'Est. Savings % (High)' (converted to decimal)




Handling Missing Data:

*  Replaces any NaN values in the new savings columns with 0
*  Ensures no null values in the estimated savings columns


Returns the final DataFrame with all original data, vendor lookup information, and calculated fields

In [ ]:
def merge_vendor_data(df, unified_lookup):
    """Merge vendor data with the original DataFrame and calculate estimated savings."""
    # Merge with unified lookup
    final_df = pd.merge(df, unified_lookup, on='Supplier Name', how='left')

    # Calculate Total if it doesn't exist
    if 'Total' not in final_df.columns:
        if 'Quantity in UOM' in final_df.columns and 'Price' in final_df.columns:
            final_df['Total'] = final_df['Quantity in UOM'] * final_df['Price']
        else:
            print("Warning: Unable to calculate Total. 'Quantity in UOM' or 'Price' column missing.")
            final_df['Total'] = np.nan

    # Calculate estimated savings
    final_df['Est. Savings (Low)'] = final_df['Total'] * (final_df['Est. Savings % (Low)'])
    final_df['Est. Savings (High)'] = final_df['Total'] * (final_df['Est. Savings % (High)'])

    # Handle potential NaN values in the new columns
    final_df['Est. Savings (Low)'] = final_df['Est. Savings (Low)'].fillna(0)
    final_df['Est. Savings (High)'] = final_df['Est. Savings (High)'].fillna(0)

    return final_df

create_missing_raw_data_report(df, output_path):

*  Identifies rows with missing data in specific columns
*  Columns checked: Supplier ID, Supplier Name, Item Description, Item Group, Product Group, UOM, Quantity, Quantity in UOM, Price
*  Creates a new DataFrame with only the rows containing missing data
*  Saves this DataFrame to an Excel file at the specified output path
*  Returns the DataFrame of missing data


create_missing_vendor_info_report(final_df, unified_lookup, output_path):

*  Identifies vendors in the client's data
*  Checks for missing information in the unified vendor lookup for these vendors
*  Columns checked: ST Category, ST Sub-Category, Opportunity Term, Est. Savings % (Low), Est. Savings % (High), Marketplace Designation
*  Creates a DataFrame of vendors with missing information
*  Saves this DataFrame to an Excel file at the specified output path
*  Returns the DataFrame of vendors with missing information


calculate_missing_data_percentage(df):

*  Calculates the percentage of missing data for each column in the DataFrame
*  Returns a sorted Series with columns and their missing data percentages


update_df_with_missing_data(original_df, missing_data):

*  Attempts to update the original DataFrame with data from a missing data file
*  Uses 'Supplier Name', 'SKU', and 'Item Description' as key columns for matching
*  Merges the original DataFrame with the missing data
*  Updates existing rows where new data is available
*  Adds completely new rows from the missing data file
*  Handles cases where key columns might be missing
*  Returns the updated DataFrame

In [ ]:
def create_missing_raw_data_report(df, output_path):
    """Create a report of missing raw data from the client's upload."""
    columns_to_check = ['Supplier ID', 'Supplier Name', 'Item Description', 'Item Group',
                        'Product Group', 'UOM', 'Quantity', 'Quantity in UOM', 'Price']

    missing_mask = df[columns_to_check].isnull().any(axis=1)
    missing_data = df[missing_mask]
    missing_data.to_excel(output_path, index=False)
    print(f"Missing raw data report saved to {output_path}")
    return missing_data

def create_missing_vendor_info_report(final_df, unified_lookup, output_path):
    """Create a report of missing vendor information for vendors in the client's data."""
    columns_to_check = ['ST Category', 'ST Sub-Category', 'Opportunity Term',
                        'Est. Savings % (Low)', 'Est. Savings % (High)', 'Marketplace Designation']

    client_vendors = final_df['Vendor ID'].unique()
    vendor_info = unified_lookup[unified_lookup['Vendor ID'].isin(client_vendors)]
    missing_mask = vendor_info[columns_to_check].isnull().any(axis=1)
    missing_vendor_info = vendor_info[missing_mask]
    missing_vendor_info.to_excel(output_path, index=False)
    print(f"Missing vendor information report saved to {output_path}")
    return missing_vendor_info

def calculate_missing_data_percentage(df):
    """Calculate the percentage of missing data for each column."""
    total_rows = len(df)
    missing_percentages = (df.isnull().sum() / total_rows * 100).round(2)
    return missing_percentages.sort_values(ascending=False)

def update_df_with_missing_data(original_df, missing_data):
    """Update the original DataFrame with data from the missing data file."""
    key_columns = ['Supplier Name', 'SKU', 'Item Description']

    for col in key_columns.copy():
        if col not in original_df.columns or col not in missing_data.columns:
            print(f"Warning: Key column '{col}' not found in one of the DataFrames. Skipping this column for matching.")
            key_columns.remove(col)

    if not key_columns:
        raise ValueError("No valid key columns for matching. Cannot update DataFrame.")

    updated_df = pd.merge(
        original_df,
        missing_data,
        on=key_columns,
        how='left',
        suffixes=('', '_new')
    )

    for col in missing_data.columns:
        if col in original_df.columns and col not in key_columns:
            mask = updated_df[f'{col}_new'].notna()
            updated_df.loc[mask, col] = updated_df.loc[mask, f'{col}_new']

    updated_df = updated_df.drop(columns=[col for col in updated_df.columns if col.endswith('_new')])

    new_rows = missing_data[~missing_data[key_columns].isin(original_df[key_columns].to_dict('list')).all(axis=1)]
    if not new_rows.empty:
        updated_df = pd.concat([updated_df, new_rows], ignore_index=True)
        print(f"Added {len(new_rows)} completely new rows from missing data.")

    return updated_df

process_file(df, column_mapping):

This function is a lower-level utility that performs specific data cleaning and transformation tasks on a DataFrame. It:

*  Renames columns based on a provided mapping
*  Handles duplicate columns
*  Converts specific columns to appropriate data types (datetime, numeric)




In [ ]:
def process_file(df, column_mapping):
    # Rename columns based on the mapping
    df_renamed = df.rename(columns={v: k for k, v in column_mapping.items()})

    # Check for duplicate columns
    duplicate_columns = df_renamed.columns[df_renamed.columns.duplicated()].tolist()
    if duplicate_columns:
        print(f"Warning: Duplicate columns found after renaming: {duplicate_columns}")
        # Append a suffix to duplicate columns
        for col in duplicate_columns:
            mask = df_renamed.columns == col
            df_renamed.columns = [f'{col}_{i}' if x else col for i, x in enumerate(mask)]

    # Perform some basic processing
    if 'Date' in df_renamed.columns:
        df_renamed['Date'] = pd.to_datetime(df_renamed['Date'], errors='coerce')

    if 'Price' in df_renamed.columns:
        df_renamed['Price'] = pd.to_numeric(df_renamed['Price'], errors='coerce')
    if 'Quantity' in df_renamed.columns:
        df_renamed['Quantity'] = pd.to_numeric(df_renamed['Quantity'], errors='coerce')
    if 'Quantity in UOM' in df_renamed.columns:
        df_renamed['Quantity in UOM'] = pd.to_numeric(df_renamed['Quantity in UOM'], errors='coerce')

    return df_renamed

process_client_input(file_path):

This is a higher-level function that orchestrates the entire process of loading and initial processing of the client's input file. It:

*  Loads the catalog
*  Extracts the client name from the file name
*  Loads the Excel workbook and finds the correct sheet
*  Calls interactive_column_mapping to get the column mapping
*  Calls process_file to apply the mapping and perform basic data cleaning
*  Adds required columns like 'Total', 'Stage', and 'Client Name'
*  Handles the case where the 'Date' column is missing

In [ ]:

def process_client_input(file_path):
    """
    Process the client's input Excel file.

    Args:
    file_path (str): The path to the client's Excel file.

    Returns:
    tuple: A tuple containing:
        - pd.DataFrame: The processed DataFrame with mapped columns and additional required columns.
        - dict: The updated catalog.
    """
    # Load the catalog
    catalog = load_or_create_catalog()

    # Extract client name from the file name
    client_name = os.path.basename(file_path).split(' - ')[0]

    # Load the workbook
    wb = load_workbook(file_path, read_only=True, data_only=True)

    # Find the sheet with 'Data' in its name
    data_sheet = next((sheet for sheet in wb.sheetnames if 'data' in sheet.lower()), None)
    if not data_sheet:
        raise ValueError("No sheet with 'Data' in its name found in the workbook.")

    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=data_sheet)

    print("Original DataFrame columns:")
    print(df.columns.tolist())

    # Perform interactive column mapping
    column_mapping, updated_catalog = interactive_column_mapping(df.columns, catalog)

    # Process the file with the new mapping
    df = process_file(df, column_mapping)

    # Filter the DataFrame to include only the columns from our catalog
    catalog_columns = list(column_mapping.keys())
    df = df[catalog_columns]

    # Add required columns
    if 'Quantity in UOM' in df.columns and 'Price' in df.columns:
        df['Total'] = df['Quantity in UOM'] * df['Price']
    else:
        print("Warning: 'Quantity in UOM' or 'Price' column not found. 'Total' column not added.")

    df['Stage'] = 'Opportunity Assessment'
    df['Client Name'] = client_name

    # If 'Date' column is missing, add it with a default value
    if 'Date' not in df.columns:
        print("Warning: 'Date' column is missing. Adding it with a default value.")
        df['Date'] = pd.Timestamp.now()

    print("\nProcessed DataFrame:")
    print(df.head())
    print("\nProcessed DataFrame columns:")
    print(df.columns.tolist())

    return df, updated_catalog

Example

In [ ]:
def run_example():
    unified_lookup_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/unified_vendor_lookup.xlsx"
    client_file_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Transcendia - Opportunity Assessment.xlsx"
    missing_raw_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_raw_data.xlsx"
    missing_vendor_info_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_vendor_info.xlsx"

    print("Starting run_example function...")

    try:
        # Load unified Vendor Look Up table
        print("Loading unified vendor lookup table...")
        unified_lookup = load_unified_vendor_lookup(unified_lookup_path)

        if unified_lookup is None:
            raise ValueError("load_unified_vendor_lookup returned None")

        print("\nUnified lookup table columns:")
        print(unified_lookup.columns.tolist())

        print("\nFirst few rows of unified lookup table:")
        print(unified_lookup.head())

        # Process client input
        print("\nProcessing client input...")
        result = process_client_input(client_file_path)

        if result is None:
            raise ValueError("process_client_input returned None")

        # Handle the case where process_client_input returns a tuple
        if isinstance(result, tuple) and len(result) == 2:
            df, updated_catalog = result
        elif isinstance(result, pd.DataFrame):
            df = result
            updated_catalog = None
        else:
            raise ValueError(f"Unexpected return type from process_client_input: {type(result)}")

        if df is None or not isinstance(df, pd.DataFrame):
            raise ValueError(f"Invalid DataFrame returned from process_client_input: {type(df)}")

        print("\nClient input DataFrame columns:")
        print(df.columns.tolist())

        print("\nFirst few rows of client input DataFrame:")
        print(df.head())

        # Create missing raw data report
        print("\nCreating missing raw data report...")
        missing_raw_data = create_missing_raw_data_report(df, missing_raw_data_path)
        print(f"Missing raw data report saved to {missing_raw_data_path}")

        # Calculate percentage of missing raw data
        raw_data_missing_percentages = calculate_missing_data_percentage(df)
        print("\nPercentage of missing data in raw data:")
        print(raw_data_missing_percentages)

        # Get unique suppliers from raw data
        new_suppliers = df['Supplier Name'].unique().tolist()

        # Update unified vendor lookup with new suppliers
        print("\nUpdating unified vendor lookup with new suppliers...")
        unified_lookup = update_vendor_lookup(unified_lookup, new_suppliers)

        # Merge vendor data
        print("\nMerging vendor data...")
        final_df = merge_vendor_data(df, unified_lookup)

        if final_df is None:
            raise ValueError("merge_vendor_data returned None")

        # Calculate estimated savings
        if 'Total' in final_df.columns and 'Est. Savings % (Low)' in final_df.columns and 'Est. Savings % (High)' in final_df.columns:
            final_df['Est. Savings (Low)'] = final_df['Total'] * (final_df['Est. Savings % (Low)'])
            final_df['Est. Savings (High)'] = final_df['Total'] * (final_df['Est. Savings % (High)'])
            print("Estimated savings calculated.")
        else:
            print("Warning: Unable to calculate estimated savings. Required columns missing.")

        print("\nFinal DataFrame columns after merging:")
        print(final_df.columns.tolist())

        print("\nFirst few rows of final DataFrame after merging:")
        print(final_df.head())

        # Create missing vendor info report
        print("\nCreating missing vendor info report...")
        missing_vendor_info = create_missing_vendor_info_report(final_df, unified_lookup, missing_vendor_info_path)
        print(f"Missing vendor information report saved to {missing_vendor_info_path}")

        # Calculate percentage of missing vendor info
        vendor_info_missing_percentages = calculate_missing_data_percentage(missing_vendor_info)
        print("\nPercentage of missing data in vendor information:")
        print(vendor_info_missing_percentages)

        # Calculate missing data percentages
        print("\nCalculating missing data percentages for final DataFrame...")
        final_df_missing_percentages = calculate_missing_data_percentage(final_df)
        print("\nPercentage of missing data in final DataFrame:")
        print(final_df_missing_percentages)

        # Identify missing suppliers
        print("\nIdentifying missing suppliers...")
        if 'Vendor ID' not in final_df.columns:
            raise KeyError("'Vendor ID' column not found in final DataFrame")

        missing_suppliers = final_df[final_df['Vendor ID'].isnull()]['Supplier Name'].unique()
        print(f"Number of missing suppliers: {len(missing_suppliers)}")

        # Add new entries for missing suppliers
        if len(missing_suppliers) > 0:
            print("\nAdding new entries for missing suppliers...")
            unified_lookup = update_vendor_lookup(unified_lookup, missing_suppliers)

            # Merge again with updated lookup
            print("\nMerging again with updated lookup...")
            final_df = merge_vendor_data(df, unified_lookup)

        print("\nFinal DataFrame shape:", final_df.shape)
        print("\nFinal DataFrame columns:")
        print(final_df.columns.tolist())
        print("\nFirst few rows of the final DataFrame:")
        print(final_df.head())

        # Calculate and display total spend and estimated savings
        if 'Total' in final_df.columns and 'Est. Savings (Low)' in final_df.columns and 'Est. Savings (High)' in final_df.columns:
            total_spend = final_df['Total'].sum()
            total_low_savings = final_df['Est. Savings (Low)'].sum()
            total_high_savings = final_df['Est. Savings (High)'].sum()

            print(f"\nTotal Spend: ${total_spend:,.2f}")
            print(f"Estimated Low Savings: ${total_low_savings:,.2f} ({total_low_savings/total_spend*100:.2f}%)")
            print(f"Estimated High Savings: ${total_high_savings:,.2f} ({total_high_savings/total_spend*100:.2f}%)")

        # Save updated unified lookup
        print("\nSaving updated unified lookup...")
        unified_lookup.to_excel(unified_lookup_path, index=False)
        print("Updated unified Vendor Look Up table saved.")

        if updated_catalog is not None:
            # Save updated catalog if it exists
            print("\nSaving updated catalog...")
            updated_catalog_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/updated_catalog.json"
            with open(updated_catalog_path, 'w') as f:
                json.dump(updated_catalog, f, indent=4)
            print(f"Updated catalog saved to: {updated_catalog_path}")

        print("\nMissing Data Summary:")
        print("Raw Data Missing Percentages:")
        print(raw_data_missing_percentages)
        print("\nVendor Info Missing Percentages:")
        print(vendor_info_missing_percentages)
        print("\nFinal DataFrame Missing Percentages:")
        print(final_df_missing_percentages)

        return final_df, unified_lookup, missing_raw_data, missing_vendor_info, final_df_missing_percentages

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None

# Run the example
if __name__ == "__main__":
    final_df, unified_lookup, missing_raw_data, missing_vendor_info, missing_percentages = run_example()

    if all(v is not None for v in [final_df, unified_lookup, missing_raw_data, missing_vendor_info, missing_percentages]):
        print("Run completed successfully")
    else:
        print("Run failed")

Starting run_example function...
Loading unified vendor lookup table...

Unified lookup table columns:
['Supplier Name', 'Parent Vendor', 'Parent Vendor ID', 'Vendor ID', 'ST Category', 'ST Sub-Category', 'Opportunity Term', 'Est. Savings % (Low)', 'Est. Savings % (High)', 'Marketplace Designation', 'Notes']

First few rows of unified lookup table:
                          Supplier Name                   Parent Vendor  \
0  419 Neon LLC                                            419 Neon LLC   
1  48Forty Solutions, LLC                        48Forty Solutions, LLC   
2  4C2 Electrical Associates, LLC        4C2 Electrical Associates, LLC   
3  A & A Machine Works, Inc.                  A & A Machine Works, Inc.   
4  A & D Flexographic Repair LLC          A & D Flexographic Repair LLC   

  Parent Vendor ID Vendor ID                      ST Category  \
0           4NL001  4NL001-1                              NaN   
1           4SL001  4SL001-1               Packaging Supplies   
2  

In [ ]:
final_df.sort_values(by='Est. Savings (Low)',ascending=False)

,Supplier ID,Supplier Name,Item Description,SKU,Item Group,Product Group,UOM,Quantity,Quantity in UOM,Price,Date,Total,Stage,Client Name,Parent Vendor,Parent Vendor ID,Vendor ID,ST Category,ST Sub-Category,Opportunity Term,Est. Savings % (Low),Est. Savings % (High),Marketplace Designation,Notes,Est. Savings (Low),Est. Savings (High)
16522,56600,"Motion Industries, Inc.",OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,18309.4300,2024-03-18,18309.43000,Opportunity Assessment,Transcendia,"Motion Industries, Inc.",MII002,MII002-1,MRO,Power Transmission,Near,0.10,0.15,C,NaN,1830.9430,2746.4145
29971,56600,"Motion Industries, Inc.",OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,9936.0000,2024-05-08,9936.00000,Opportunity Assessment,Transcendia,"Motion Industries, Inc.",MII002,MII002-1,MRO,Power Transmission,Near,0.10,0.15,C,NaN,993.6000,1490.4000
29901,56600,"Motion Industries, Inc.",OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,7890.4300,2024-02-02,7890.43000,Opportunity Assessment,Transcendia,"Motion Industries, Inc.",MII002,MII002-1,MRO,Power Transmission,Near,0.10,0.15,C,NaN,789.0430,1183.5645
17361,54650,McMaster-Carr,OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,5512.3100,2024-06-27,5512.31000,Opportunity Assessment,Transcendia,McMaster-Carr,M003,M003-1,MRO,General Industrial Supplies,Near,0.12,0.17,C,NaN,661.4772,937.0927
26937,56600,"Motion Industries, Inc.",OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,6297.8500,2023-11-16,6297.85000,Opportunity Assessment,Transcendia,"Motion Industries, Inc.",MII002,MII002-1,MRO,Power Transmission,Near,0.10,0.15,C,NaN,629.7850,944.6775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33019,88726,Jack Daggy Flowers,OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,93.0900,2023-09-06,93.09000,Opportunity Assessment,Transcendia,Jack Daggy Flowers,JDF001,JDF001-1,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
33023,70375,Richmond Hydraulic Service Inc.,OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,5.67,5.67,6.5500,2024-07-09,37.13850,Opportunity Assessment,Transcendia,Richmond Hydraulic Service Inc.,RHS001,RHS001-1,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
33024,63615,"PC Connections, Inc.",OPERATION SUPPLIES,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,329.0000,2024-05-31,329.00000,Opportunity Assessment,Transcendia,"PC Connections, Inc.",PCI001,PCI001-1,NaN,NaN,NaN,NaN,NaN,TBD,NaN,NaN,NaN
33025,14991,Dayton Barns and All Steel Buildings,SERVICES-NON MATERIAL,,SUBCONTRACTED ITEMS GROUP,MISCELLANEOUS,EA,1.00,1.00,205.0100,2024-05-21,205.01000,Opportunity Assessment,Transcendia,Dayton Barns and All Steel Buildings,DBA001,DBA001-1,NaN,NaN,NaN,NaN,NaN,TBD,NaN,NaN,NaN


App

In [ ]:
final_df.to_csv('final_df.csv', index=False)


In [ ]:
!pip install streamlit plotly
!pip install streamlit pyngrok
from pyngrok import ngrok

In [ ]:

%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


# Set page configuration
st.set_page_config(page_title="Opportunity Savings Dashboard", layout="wide")

# Custom CSS to inject
st.markdown("""
    <style>
    .big-font {
        font-size:30px !important;
        font-weight: bold;
    }
    .medium-font {
        font-size:20px !important;
    }
    .small-font {
        font-size:14px !important;
    }
    .metric-card {
        background-color: #f0f2f6;
        border-radius: 10px;
        padding: 15px;
        text-align: center;
    }
    </style>
    """, unsafe_allow_html=True)

@st.cache_data
def load_data():
    df = pd.read_csv('final_df.csv')
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])  # Remove rows with invalid dates

    # Convert 'ST Category' to string and replace NaN with 'Uncategorized'
    df['ST Category'] = df['ST Category'].fillna('Uncategorized').astype(str)

    return df

def calculate_metrics(df):
    total_spend = df['Total'].sum()
    total_low_savings = df['Est. Savings (Low)'].sum()
    total_high_savings = df['Est. Savings (High)'].sum()
    avg_low_savings_pct = (total_low_savings / total_spend) * 100
    avg_high_savings_pct = (total_high_savings / total_spend) * 100
    return total_spend, total_low_savings, total_high_savings, avg_low_savings_pct, avg_high_savings_pct

def display_savings_table(df, title, is_overall=True):
    st.subheader(title)
    st.table(df)

    if is_overall:
        # Pie chart code remains the same
        df_pie = df[df['ST Category'] != 'Grand Total']
        fig = px.pie(df_pie,
                     values='Est. Savings $ (High)',
                     names='ST Category',
                     title=f"{title} - High Savings Distribution",
                     hover_data=['Est Savings % (High)'],
                     labels={'Est. Savings $ (High)': 'High Savings $'})
        fig.update_traces(textposition='inside', textinfo='percent+label')
        fig.update_layout(height=500)
    else:
        # Horizontal stacked bar chart for detailed high savings by sub-category
        df_plot = df[(df['ST Category'] != 'Grand Total') &
                     (df['ST Sub-Category / Parent Vendor'] != 'Grand Total')].copy()

        # Remove rows where Sub-Category ends with ' Total'
        df_plot = df_plot[~df_plot['ST Sub-Category / Parent Vendor'].astype(str).str.endswith(' Total')]

        # Sort categories by total savings
        category_order = df_plot.groupby('ST Category')['Est. Savings $ (High)'].sum().sort_values(ascending=True).index

        fig = px.bar(df_plot,
                     x='Est. Savings $ (High)',
                     y='ST Category',
                     color='ST Sub-Category / Parent Vendor',
                     orientation='h',
                     title=f"{title} - High Savings by Category and Sub-Category",
                     labels={'Est. Savings $ (High)': 'High Savings ($)', 'ST Category': 'Category', 'ST Sub-Category / Parent Vendor': 'Sub-Category'},
                     category_orders={'ST Category': category_order})

        fig.update_layout(
            barmode='stack',
            height=800,
            yaxis={'categoryorder':'array', 'categoryarray':category_order},
            legend_title_text='Sub-Category'
        )

    st.plotly_chart(fig, use_container_width=True)

def display_monthly_savings(df):
    st.subheader("Monthly Savings Trend")

    # Ensure 'Date' column is datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Group by month and calculate total savings
    monthly_savings = df.groupby(df['Date'].dt.to_period('M')).agg({
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).reset_index()
    monthly_savings['Date'] = monthly_savings['Date'].dt.to_timestamp()

    # Create line chart
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=monthly_savings['Date'], y=monthly_savings['Est. Savings (Low)'],
                             mode='lines+markers', name='Low Savings'))
    fig.add_trace(go.Scatter(x=monthly_savings['Date'], y=monthly_savings['Est. Savings (High)'],
                             mode='lines+markers', name='High Savings'))

    fig.update_layout(
        title="Monthly Savings Trend",
        xaxis_title="Month",
        yaxis_title="Savings ($)",
        height=500
    )

    st.plotly_chart(fig, use_container_width=True)

def create_savings_tables(df):
    # Table 1: Overall Savings
    overall_savings = df.groupby('ST Category').agg({
        'Total': 'sum',
        'Est. Savings % (Low)': 'mean',
        'Est. Savings % (High)': 'mean',
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).reset_index()

    overall_savings = overall_savings.rename(columns={
        'Total': 'Near Term Spend',
        'Est. Savings % (Low)': 'Est Savings % (Low)',
        'Est. Savings % (High)': 'Est Savings % (High)',
        'Est. Savings (Low)': 'Est Savings $ (Low)',
        'Est. Savings (High)': 'Est. Savings $ (High)'
    })

    # Add Grand Total row (calculated from the original dataframe)
    grand_total = df.agg({
        'Total': 'sum',
        'Est. Savings % (Low)': 'mean',
        'Est. Savings % (High)': 'mean',
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).to_frame().T
    grand_total['ST Category'] = 'Grand Total'
    grand_total = grand_total.rename(columns={
        'Total': 'Near Term Spend',
        'Est. Savings % (Low)': 'Est Savings % (Low)',
        'Est. Savings % (High)': 'Est Savings % (High)',
        'Est. Savings (Low)': 'Est Savings $ (Low)',
        'Est. Savings (High)': 'Est. Savings $ (High)'
    })
    overall_savings = pd.concat([overall_savings, grand_total], ignore_index=True)

    # Table 2: Detailed Savings
    detailed_savings = df.groupby(['ST Category', 'ST Sub-Category']).agg({
        'Total': 'sum',
        'Est. Savings % (Low)': 'mean',
        'Est. Savings % (High)': 'mean',
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).reset_index()

    detailed_savings = detailed_savings.rename(columns={
        'ST Sub-Category': 'ST Sub-Category / Parent Vendor',
        'Total': 'Near Term Spend',
        'Est. Savings % (Low)': 'Est Savings % (Low)',
        'Est. Savings % (High)': 'Est Savings % (High)',
        'Est. Savings (Low)': 'Est Savings $ (Low)',
        'Est. Savings (High)': 'Est. Savings $ (High)'
    })

    # Add Total rows for each category
    category_totals = df.groupby('ST Category').agg({
        'Total': 'sum',
        'Est. Savings % (Low)': 'mean',
        'Est. Savings % (High)': 'mean',
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).reset_index()
    category_totals['ST Sub-Category / Parent Vendor'] = 'Total'
    category_totals = category_totals.rename(columns={
        'Total': 'Near Term Spend',
        'Est. Savings % (Low)': 'Est Savings % (Low)',
        'Est. Savings % (High)': 'Est Savings % (High)',
        'Est. Savings (Low)': 'Est Savings $ (Low)',
        'Est. Savings (High)': 'Est. Savings $ (High)'
    })
    detailed_savings = pd.concat([detailed_savings, category_totals], ignore_index=True)

    # Add Grand Total row for detailed savings (same as overall grand total)
    detailed_savings = pd.concat([detailed_savings, grand_total], ignore_index=True)

    return overall_savings, detailed_savings

def display_savings_vs_spend(df):
    st.subheader("Spend vs Savings Opportunity")

    # Group by ST Category to aggregate the spend and savings data
    category_data = df.groupby('ST Category').agg({
        'Total': 'sum',
        'Est. Savings (High)': 'sum',
        'Est. Savings % (High)': 'mean'
    }).reset_index()

    # Calculate the relative size for bubbles
    category_data['Savings Ratio'] = category_data['Est. Savings (High)'] / category_data['Total'] * 100

    # Plot the bubble chart
    fig = px.scatter(category_data,
                     x='Total',
                     y='Est. Savings (High)',
                     size='Savings Ratio',
                     color='ST Category',
                     hover_data={'Total': ':.2f', 'Est. Savings (High)': ':.2f', 'Savings Ratio': ':.2f'},
                     labels={'Total': 'Total Spend ($)', 'Est. Savings (High)': 'Potential Savings ($)', 'Savings Ratio': 'Savings Ratio (%)'},
                     title="Total Spend vs. Potential Savings by Category",
                     size_max=60)

    fig.update_layout(height=600, xaxis_title="Total Spend ($)", yaxis_title="Potential High Savings ($)")
    st.plotly_chart(fig, use_container_width=True)



def main():
    st.markdown("<h1 style='text-align: center;'>Opportunity Savings Dashboard</h1>", unsafe_allow_html=True)

    # Load data
    df = load_data()

    # Sidebar filters
    st.sidebar.title("Filters")

    # Sort categories, handling potential mixed types
    categories = ['All'] + sorted(df['ST Category'].unique().tolist(), key=str)
    selected_category = st.sidebar.selectbox("Select Category", categories)

    if 'Date' in df.columns and not df['Date'].empty:
        min_date = df['Date'].min().date()
        max_date = df['Date'].max().date()
        date_range = st.sidebar.date_input("Select Date Range",
                                           value=[min_date, max_date],
                                           min_value=min_date,
                                           max_value=max_date)

        # Apply filters
        if selected_category != 'All':
            df = df[df['ST Category'] == selected_category]
        if len(date_range) == 2:
            df = df[(df['Date'].dt.date >= date_range[0]) & (df['Date'].dt.date <= date_range[1])]
    else:
        st.warning("Date column is missing or empty. Date filtering is disabled.")
        if selected_category != 'All':
            df = df[df['ST Category'] == selected_category]

    # Calculate metrics
    total_spend, total_low_savings, total_high_savings, avg_low_savings_pct, avg_high_savings_pct = calculate_metrics(df)

    # Display high-level metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.markdown('<p class="medium-font">Total Spend</p>', unsafe_allow_html=True)
        st.markdown(f'<p class="big-font">${total_spend:,.0f}</p>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

    with col2:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.markdown('<p class="medium-font">Est. Low Savings</p>', unsafe_allow_html=True)
        st.markdown(f'<p class="big-font">${total_low_savings:,.0f}</p>', unsafe_allow_html=True)
        st.markdown(f'<p class="small-font">({avg_low_savings_pct:.2f}%)</p>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

    with col3:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.markdown('<p class="medium-font">Est. High Savings</p>', unsafe_allow_html=True)
        st.markdown(f'<p class="big-font">${total_high_savings:,.0f}</p>', unsafe_allow_html=True)
        st.markdown(f'<p class="small-font">({avg_high_savings_pct:.2f}%)</p>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

    st.markdown("<br>", unsafe_allow_html=True)

    # Visualizations
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Savings by Category")
        category_savings = df.groupby('ST Category')[['Est. Savings (Low)', 'Est. Savings (High)', 'Total']].sum().reset_index()
        fig = px.bar(category_savings, x='ST Category', y=['Est. Savings (Low)', 'Est. Savings (High)'],
                     title="Estimated Savings by Category", barmode='group')
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        st.subheader("Top 10 Suppliers by Potential Savings")
        top_suppliers = df.groupby('Supplier Name')['Est. Savings (High)'].sum().nlargest(10).reset_index()
        fig = px.bar(top_suppliers, x='Supplier Name', y='Est. Savings (High)',
                     title="Top 10 Suppliers by Potential High Savings")
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

    # Display monthly savings trend
    display_monthly_savings(df)

    display_savings_vs_spend(df)
    # Heatmap
    if 'Date' in df.columns and not df['Date'].empty:
        st.subheader("Savings Heatmap by Category and Month")
        df['Month'] = df['Date'].dt.to_period('M').astype(str)
        heatmap_data = df.pivot_table(values='Est. Savings (High)', index='ST Category', columns='Month', aggfunc='sum')
        fig = px.imshow(heatmap_data, aspect="auto", color_continuous_scale="YlOrRd")
        fig.update_layout(height=500)
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("Date information is not available. Heatmap cannot be displayed.")

   # Treemap
    st.subheader("Spend Distribution Treemap")
    # Filter out zero or negative values for the treemap
    df_treemap = df[df['Total'] > 0]
    if not df_treemap.empty:
        fig = px.treemap(df_treemap, path=['ST Category', 'Supplier Name'], values='Total',
                         color='Est. Savings (High)', color_continuous_scale='RdYlGn_r',
                         title="Spend Distribution and Savings Potential")
        fig.update_layout(height=600)
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("No positive 'Total' values available for the treemap visualization.")


    st.markdown("<br>", unsafe_allow_html=True)
    st.markdown("## Savings Estimates Tables", unsafe_allow_html=True)

    # Create and display savings tables
    overall_savings, detailed_savings = create_savings_tables(df)
    display_savings_table(overall_savings, "Transcendia - Wave 1 Savings Estimates", is_overall=True)
    display_savings_table(detailed_savings, "Detailed Savings Estimates", is_overall=False)

    st.markdown("<br>", unsafe_allow_html=True)
    st.markdown("## Raw Data", unsafe_allow_html=True)

    # Display raw data
    st.subheader("Raw Data")
    # Display only the first 1000 rows to prevent performance issues
    st.dataframe(df.head(1000))
    if len(df) > 1000:
        st.info(f"Showing first 1000 rows out of {len(df)} total rows.")

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2nCrHcKsJCDT2SiVXRa56Z2LMkz_3uVedRt8Q5oT5BPb9R9Lx")

# Set up a tunnel to the full address with http protocol
public_url = ngrok.connect(addr="http://localhost:8501")  # Full address
print("Public URL:", public_url)

# Run the Streamlit app
!streamlit run app.py &


Public URL: NgrokTunnel: "https://03b8-34-125-62-150.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.62.150:8501

  Stopping...


Missing data


In [ ]:
def update_with_missing_data(existing_df, missing_data_file):
    """
    Update the existing DataFrame with data from the missing data file.

    Args:
    existing_df (pd.DataFrame): The current DataFrame with potentially missing data.
    missing_data_file (str): Path to the Excel file containing the missing data.

    Returns:
    pd.DataFrame: Updated DataFrame with filled-in missing data.
    """
    # Load the missing data file
    missing_df = pd.read_excel(missing_data_file)

    # Ensure the 'Supplier Name' and 'SKU' columns exist in both DataFrames
    key_columns = ['Supplier Name', 'SKU']
    if not all(col in existing_df.columns for col in key_columns) or \
       not all(col in missing_df.columns for col in key_columns):
        raise ValueError("Both DataFrames must contain 'Supplier Name' and 'SKU' columns")

    # Merge the DataFrames
    updated_df = existing_df.merge(missing_df, on=key_columns, how='left', suffixes=('', '_new'))

    # Update the existing DataFrame with new data where applicable
    for col in missing_df.columns:
        if col not in key_columns:
            mask = updated_df[f'{col}_new'].notna()
            updated_df.loc[mask, col] = updated_df.loc[mask, f'{col}_new']

    # Drop the temporary columns created by the merge
    columns_to_drop = [col for col in updated_df.columns if col.endswith('_new')]
    updated_df = updated_df.drop(columns=columns_to_drop)

    return updated_df

run again

In [ ]:
def run_example():
    unified_lookup_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/unified_vendor_lookup.xlsx"
    client_file_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Transcendia - Opportunity Assessment.xlsx"
    missing_raw_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_raw_data.xlsx"
    missing_vendor_info_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_vendor_info.xlsx"
    filled_missing_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/filled_missing_data.xlsx"

    print("Starting run_example function...")

    try:
        # Load unified Vendor Look Up table
        print("Loading unified vendor lookup table...")
        unified_lookup = load_unified_vendor_lookup(unified_lookup_path)

        if unified_lookup is None:
            raise ValueError("load_unified_vendor_lookup returned None")

        print("\nUnified lookup table columns:")
        print(unified_lookup.columns.tolist())

        print("\nFirst few rows of unified lookup table:")
        print(unified_lookup.head())

        # Process client input
        print("\nProcessing client input...")
        result = process_client_input(client_file_path)

        if result is None:
            raise ValueError("process_client_input returned None")

        if isinstance(result, tuple) and len(result) == 2:
            df, updated_catalog = result
        elif isinstance(result, pd.DataFrame):
            df = result
            updated_catalog = None
        else:
            raise ValueError(f"Unexpected return type from process_client_input: {type(result)}")

        if df is None or not isinstance(df, pd.DataFrame):
            raise ValueError(f"Invalid DataFrame returned from process_client_input: {type(df)}")

        print("\nClient input DataFrame columns:")
        print(df.columns.tolist())

        print("\nFirst few rows of client input DataFrame:")
        print(df.head())

        # Create missing raw data report
        print("\nCreating missing raw data report...")
        missing_raw_data = create_missing_raw_data_report(df, missing_raw_data_path)
        print(f"Missing raw data report saved to {missing_raw_data_path}")

        # Calculate percentage of missing raw data
        raw_data_missing_percentages = calculate_missing_data_percentage(df)
        print("\nPercentage of missing data in raw data:")
        print(raw_data_missing_percentages)

        # Get unique suppliers from raw data
        new_suppliers = df['Supplier Name'].unique().tolist()

        # Update unified vendor lookup with new suppliers
        print("\nUpdating unified vendor lookup with new suppliers...")
        unified_lookup = update_vendor_lookup(unified_lookup, new_suppliers)

        # Merge vendor data
        print("\nMerging vendor data...")
        final_df = merge_vendor_data(df, unified_lookup)

        if final_df is None:
            raise ValueError("merge_vendor_data returned None")

        # Calculate estimated savings
        if 'Total' in final_df.columns and 'Est. Savings % (Low)' in final_df.columns and 'Est. Savings % (High)' in final_df.columns:
            final_df['Est. Savings (Low)'] = final_df['Total'] * (final_df['Est. Savings % (Low)'])
            final_df['Est. Savings (High)'] = final_df['Total'] * (final_df['Est. Savings % (High)'])
            print("Estimated savings calculated.")
        else:
            print("Warning: Unable to calculate estimated savings. Required columns missing.")

        print("\nFinal DataFrame columns after merging:")
        print(final_df.columns.tolist())

        print("\nFirst few rows of final DataFrame after merging:")
        print(final_df.head())

        # New section: Update with filled missing data
        print("\nChecking for filled missing data file...")
        if os.path.exists(filled_missing_data_path):
            print(f"Filled missing data file found: {filled_missing_data_path}")

            # Store the columns with null values before updating
            null_columns = final_df.columns[final_df.isnull().any()].tolist()

            print("Updating final DataFrame with filled missing data...")
            final_df_before = final_df.copy()
            final_df = update_with_missing_data(final_df, filled_missing_data_path)
            print("Final DataFrame updated with filled missing data.")

            # Verify the update process
            print("\nVerifying the update process...")
            try:
                # Ensure both DataFrames have the same index and columns
                common_index = final_df.index.intersection(final_df_before.index)
                common_columns = list(set(null_columns) & set(final_df.columns) & set(final_df_before.columns))

                if not common_columns:
                    print("No common columns found for comparison.")
                else:
                    final_df_subset = final_df.loc[common_index, common_columns]
                    final_df_before_subset = final_df_before.loc[common_index, common_columns]

                    # Compare the subsets
                    updated_mask = (final_df_subset != final_df_before_subset) & ~final_df_subset.isnull()
                    updated_rows = final_df_subset[updated_mask.any(axis=1)]

                    if not updated_rows.empty:
                        print(f"\nFirst 5 rows of previously null data that is now filled:")
                        print(updated_rows.head())
                    else:
                        print("No previously null values were updated.")
            except Exception as e:
                print(f"Error during verification: {str(e)}")
                print("Proceeding with the updated data without verification.")

            # Recalculate metrics after updating
            print("\nRecalculating metrics after updating with filled missing data...")
            if 'Total' in final_df.columns and 'Est. Savings % (Low)' in final_df.columns and 'Est. Savings % (High)' in final_df.columns:
                final_df['Est. Savings (Low)'] = final_df['Total'] * (final_df['Est. Savings % (Low)'])
                final_df['Est. Savings (High)'] = final_df['Total'] * (final_df['Est. Savings % (High)'])
                print("Estimated savings recalculated.")
            else:
                print("Warning: Unable to recalculate estimated savings. Required columns missing.")
        else:
            print("No filled missing data file found. Proceeding with original data.")

        # Create missing vendor info report
        print("\nCreating missing vendor info report...")
        missing_vendor_info = create_missing_vendor_info_report(final_df, unified_lookup, missing_vendor_info_path)
        print(f"Missing vendor information report saved to {missing_vendor_info_path}")

        # Calculate percentage of missing vendor info
        vendor_info_missing_percentages = calculate_missing_data_percentage(missing_vendor_info)
        print("\nPercentage of missing data in vendor information:")
        print(vendor_info_missing_percentages)

        # Calculate missing data percentages
        print("\nCalculating missing data percentages for final DataFrame...")
        final_df_missing_percentages = calculate_missing_data_percentage(final_df)
        print("\nPercentage of missing data in final DataFrame:")
        print(final_df_missing_percentages)

        # Calculate and display total spend and estimated savings
        if 'Total' in final_df.columns and 'Est. Savings (Low)' in final_df.columns and 'Est. Savings (High)' in final_df.columns:
            total_spend = final_df['Total'].sum()
            total_low_savings = final_df['Est. Savings (Low)'].sum()
            total_high_savings = final_df['Est. Savings (High)'].sum()

            print(f"\nTotal Spend: ${total_spend:,.2f}")
            print(f"Estimated Low Savings: ${total_low_savings:,.2f} ({total_low_savings/total_spend*100:.2f}%)")
            print(f"Estimated High Savings: ${total_high_savings:,.2f} ({total_high_savings/total_spend*100:.2f}%)")

        # Save updated unified lookup
        print("\nSaving updated unified lookup...")
        unified_lookup.to_excel(unified_lookup_path, index=False)
        print("Updated unified Vendor Look Up table saved.")

        if updated_catalog is not None:
            # Save updated catalog if it exists
            print("\nSaving updated catalog...")
            updated_catalog_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/updated_catalog.json"
            with open(updated_catalog_path, 'w') as f:
                json.dump(updated_catalog, f, indent=4)
            print(f"Updated catalog saved to: {updated_catalog_path}")

        print("\nMissing Data Summary:")
        print("Raw Data Missing Percentages:")
        print(raw_data_missing_percentages)
        print("\nVendor Info Missing Percentages:")
        print(vendor_info_missing_percentages)
        print("\nFinal DataFrame Missing Percentages:")
        print(final_df_missing_percentages)

        return final_df, unified_lookup, missing_raw_data, missing_vendor_info, final_df_missing_percentages

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None

# Run the example
if __name__ == "__main__":
    final_df, unified_lookup, missing_raw_data, missing_vendor_info, missing_percentages = run_example()

    if all(v is not None for v in [final_df, unified_lookup, missing_raw_data, missing_vendor_info, missing_percentages]):
        print("Run completed successfully")
    else:
        print("Run failed")

Starting run_example function...
Loading unified vendor lookup table...

Unified lookup table columns:
['Supplier Name', 'Parent Vendor', 'Parent Vendor ID', 'Vendor ID', 'ST Category', 'ST Sub-Category', 'Opportunity Term', 'Est. Savings % (Low)', 'Est. Savings % (High)', 'Marketplace Designation', 'Notes']

First few rows of unified lookup table:
                          Supplier Name                   Parent Vendor  \
0  419 Neon LLC                                            419 Neon LLC   
1  48Forty Solutions, LLC                        48Forty Solutions, LLC   
2  4C2 Electrical Associates, LLC        4C2 Electrical Associates, LLC   
3  A & A Machine Works, Inc.                  A & A Machine Works, Inc.   
4  A & D Flexographic Repair LLC          A & D Flexographic Repair LLC   

  Parent Vendor ID Vendor ID                      ST Category  \
0           4NL001  4NL001-1                              NaN   
1           4SL001  4SL001-1               Packaging Supplies   
2  

KeyboardInterrupt: 

All Opp Assesment Master File

In [ ]:
def run_example(client_file_path, unified_lookup_path, master_table_path):
    missing_raw_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_raw_data.xlsx"
    missing_vendor_info_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_vendor_info.xlsx"
    filled_missing_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/filled_missing_data.xlsx"

    print(f"Starting run_example function for file: {client_file_path}")

    try:
        # Load unified Vendor Look Up table
        print("Loading unified vendor lookup table...")
        unified_lookup = load_unified_vendor_lookup(unified_lookup_path)

        if unified_lookup is None:
            raise ValueError("load_unified_vendor_lookup returned None")

        print("\nUnified lookup table columns:")
        print(unified_lookup.columns.tolist())

        print("\nFirst few rows of unified lookup table:")
        print(unified_lookup.head())

        # Process client input
        print("\nProcessing client input...")
        result = process_client_input(client_file_path)

        if isinstance(result, tuple) and len(result) == 2:
            df, updated_catalog = result
        elif isinstance(result, pd.DataFrame):
            df = result
            updated_catalog = None
        else:
            raise ValueError(f"Unexpected return type from process_client_input: {type(result)}")

        if df is None or not isinstance(df, pd.DataFrame):
            raise ValueError(f"Invalid DataFrame returned from process_client_input: {type(df)}")

        print("\nClient input DataFrame columns:")
        print(df.columns.tolist())

        print("\nFirst few rows of client input DataFrame:")
        print(df.head())

        # Create missing raw data report
        print("\nCreating missing raw data report...")
        missing_raw_data = create_missing_raw_data_report(df, missing_raw_data_path)
        print(f"Missing raw data report saved to {missing_raw_data_path}")

        # Calculate percentage of missing raw data
        raw_data_missing_percentages = calculate_missing_data_percentage(df)
        print("\nPercentage of missing data in raw data:")
        print(raw_data_missing_percentages)

        # Get unique suppliers from raw data
        new_suppliers = df['Supplier Name'].unique().tolist()

        # Update unified vendor lookup with new suppliers
        print("\nUpdating unified vendor lookup with new suppliers...")
        unified_lookup = update_vendor_lookup(unified_lookup, new_suppliers)

        # Merge vendor data
        print("\nMerging vendor data...")
        final_df = merge_vendor_data(df, unified_lookup)

        if final_df is None:
            raise ValueError("merge_vendor_data returned None")

        # Calculate estimated savings
        if 'Total' in final_df.columns and 'Est. Savings % (Low)' in final_df.columns and 'Est. Savings % (High)' in final_df.columns:
            final_df['Est. Savings (Low)'] = final_df['Total'] * (final_df['Est. Savings % (Low)'])
            final_df['Est. Savings (High)'] = final_df['Total'] * (final_df['Est. Savings % (High)'])
            print("Estimated savings calculated.")
        else:
            print("Warning: Unable to calculate estimated savings. Required columns missing.")

        print("\nFinal DataFrame columns after merging:")
        print(final_df.columns.tolist())

        print("\nFirst few rows of final DataFrame after merging:")
        print(final_df.head())

        # Update with filled missing data
        print("\nChecking for filled missing data file...")
        if os.path.exists(filled_missing_data_path):
            print(f"Filled missing data file found: {filled_missing_data_path}")

            # Store the columns with null values before updating
            null_columns = final_df.columns[final_df.isnull().any()].tolist()

            print("Updating final DataFrame with filled missing data...")
            final_df_before = final_df.copy()
            final_df = update_with_missing_data(final_df, filled_missing_data_path)
            print("Final DataFrame updated with filled missing data.")

            # Verify the update process
            print("\nVerifying the update process...")
            try:
                # Ensure both DataFrames have the same index and columns
                common_index = final_df.index.intersection(final_df_before.index)
                common_columns = list(set(null_columns) & set(final_df.columns) & set(final_df_before.columns))

                if not common_columns:
                    print("No common columns found for comparison.")
                else:
                    final_df_subset = final_df.loc[common_index, common_columns]
                    final_df_before_subset = final_df_before.loc[common_index, common_columns]

                    # Compare the subsets
                    updated_mask = (final_df_subset != final_df_before_subset) & ~final_df_subset.isnull()
                    updated_rows = final_df_subset[updated_mask.any(axis=1)]

                    if not updated_rows.empty:
                        print(f"\nFirst 5 rows of previously null data that is now filled:")
                        print(updated_rows.head())
                    else:
                        print("No previously null values were updated.")
            except Exception as e:
                print(f"Error during verification: {str(e)}")
                print("Proceeding with the updated data without verification.")

            # Recalculate metrics after updating
            print("\nRecalculating metrics after updating with filled missing data...")
            if 'Total' in final_df.columns and 'Est. Savings % (Low)' in final_df.columns and 'Est. Savings % (High)' in final_df.columns:
                final_df['Est. Savings (Low)'] = final_df['Total'] * (final_df['Est. Savings % (Low)'])
                final_df['Est. Savings (High)'] = final_df['Total'] * (final_df['Est. Savings % (High)'])
                print("Estimated savings recalculated.")
            else:
                print("Warning: Unable to recalculate estimated savings. Required columns missing.")
        else:
            print("No filled missing data file found. Proceeding with original data.")

        # Create missing vendor info report
        print("\nCreating missing vendor info report...")
        missing_vendor_info = create_missing_vendor_info_report(final_df, unified_lookup, missing_vendor_info_path)
        print(f"Missing vendor information report saved to {missing_vendor_info_path}")

        # Calculate percentage of missing vendor info
        vendor_info_missing_percentages = calculate_missing_data_percentage(missing_vendor_info)
        print("\nPercentage of missing data in vendor information:")
        print(vendor_info_missing_percentages)

        # Calculate missing data percentages
        print("\nCalculating missing data percentages for final DataFrame...")
        final_df_missing_percentages = calculate_missing_data_percentage(final_df)
        print("\nPercentage of missing data in final DataFrame:")
        print(final_df_missing_percentages)

        # Calculate and display total spend and estimated savings
        if 'Total' in final_df.columns and 'Est. Savings (Low)' in final_df.columns and 'Est. Savings (High)' in final_df.columns:
            total_spend = final_df['Total'].sum()
            total_low_savings = final_df['Est. Savings (Low)'].sum()
            total_high_savings = final_df['Est. Savings (High)'].sum()

            print(f"\nTotal Spend: ${total_spend:,.2f}")
            print(f"Estimated Low Savings: ${total_low_savings:,.2f} ({total_low_savings/total_spend*100:.2f}%)")
            print(f"Estimated High Savings: ${total_high_savings:,.2f} ({total_high_savings/total_spend*100:.2f}%)")

        # Extract client name from the file name
        client_name = os.path.basename(client_file_path).split(' - ')[0]

        # Update master table
        print("\nUpdating master table...")
        master_table = update_master_table(final_df, client_name, master_table_path)
        print("Master table updated successfully.")

        # Save updated unified lookup
        print("\nSaving updated unified lookup...")
        unified_lookup.to_excel(unified_lookup_path, index=False)
        print("Updated unified Vendor Look Up table saved.")

        if updated_catalog is not None:
            # Save updated catalog if it exists
            print("\nSaving updated catalog...")
            updated_catalog_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/updated_catalog.json"
            with open(updated_catalog_path, 'w') as f:
                json.dump(updated_catalog, f, indent=4)
            print(f"Updated catalog saved to: {updated_catalog_path}")

        print("\nMissing Data Summary:")
        print("Raw Data Missing Percentages:")
        print(raw_data_missing_percentages)
        print("\nVendor Info Missing Percentages:")
        print(vendor_info_missing_percentages)
        print("\nFinal DataFrame Missing Percentages:")
        print(final_df_missing_percentages)

        return final_df, unified_lookup, missing_raw_data, missing_vendor_info, final_df_missing_percentages, master_table

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None, None

# Function to process multiple client files
def process_multiple_clients(client_files, unified_lookup_path, master_table_path):
    for client_file in client_files:
        print(f"\nProcessing file: {client_file}")
        final_df, unified_lookup, _, _, _, master_table = run_example(client_file, unified_lookup_path, master_table_path)
        if final_df is None:
            print(f"Failed to process {client_file}")
        else:
            print(f"Successfully processed {client_file}")

    print(f"\nAll client files processed. Master table saved to {master_table_path}")
    return master_table

# Example usage
if __name__ == "__main__":
    client_folder = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Client Files"
    unified_lookup_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/unified_vendor_lookup.xlsx"
    master_table_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/opportunity_assessments_master_table.xlsx"

    client_files = [os.path.join(client_folder, f) for f in os.listdir(client_folder) if f.endswith('.xlsx')]
    master_table = process_multiple_clients(client_files, unified_lookup_path, master_table_path)

    print(f"Master table shape: {master_table.shape}")


# Ingredients

Catalog

In [ ]:
def load_or_create_catalog_ingredients(catalog_file='column_name_catalog.json'):
    if os.path.exists(catalog_file):
        with open(catalog_file, 'r') as f:
            return json.load(f)
    else:
        return {
            'Item ID': ['Item ID', 'Item Number', 'Item code','Itemcode','SKU'],
            'Supplier Name': ['Supplier Name', 'Vendor Name', 'Supplier', 'Vendor','Vendor','Vendor Name (PO)'],
            'Item Description': ['Item Description', 'Product Description', 'Description', 'Item Name', 'Product Name'],
            'UOM': ['UOM', 'Unit of Measure', 'Purchase UOM', 'Order UOM',' Order UOM '],
            'Quantity in UOM': ['Received Quantity','Qty Received In Purch UOM', 'Order Qty UOM', 'Quantity in Purchase UOM','Quatity UOM','Quantity'],
            'Total': ['Actual Inventory Cost'],
            'Date': ['Received Date','Receive Date', 'Purchase Date', 'Order Date', 'Transaction Date', 'Invoice Date']
        }
def save_catalog(catalog, catalog_file='column_name_catalog.json'):
    """
    Save the catalog to a JSON file.

    Args:
    catalog (dict): The catalog dictionary to be saved.
    catalog_file (str): The filename where the catalog will be saved.
    """
    with open(catalog_file, 'w') as f:
        json.dump(catalog, f, indent=4)

def find_matching_column(df_columns, catalog):
    column_mapping = {}
    unmatched_columns = []
    df_columns = pd.Index(df_columns)

    for standard_name, variations in catalog.items():
        match_found = False
        for possible_name in variations:
            exact_match = df_columns[df_columns.str.lower() == possible_name.lower()]
            if len(exact_match) > 0:
                column_mapping[standard_name] = exact_match[0]
                match_found = True
                break

            partial_matches = [col for col in df_columns if possible_name.lower() in col.lower()]
            if partial_matches:
                column_mapping[standard_name] = partial_matches[0]
                match_found = True
                break

        if not match_found:
            unmatched_columns.append(standard_name)

    return column_mapping, unmatched_columns
def update_catalog(catalog, new_mappings):
    """
    Update the existing catalog with new column name mappings.

    Args:
    catalog (dict): The existing catalog dictionary.
    new_mappings (dict): A dictionary of new mappings to add or update.

    Returns:
    dict: The updated catalog dictionary.
    """
    for standard_name, variations in new_mappings.items():
        if standard_name in catalog:
            catalog[standard_name].extend([v for v in variations if v not in catalog[standard_name]])
        else:
            catalog[standard_name] = variations
    return catalog

def interactive_column_mapping_ingredients(df_columns, catalog):
    column_mapping, unmatched_columns = find_matching_column(df_columns, catalog)

    print("Initial column mapping:")
    for standard_name, actual_name in column_mapping.items():
        print(f"{standard_name}: {actual_name}")

    print("\nUnmatched columns:")
    for col in unmatched_columns:
        print(f"No match found for '{col}'")

    new_mappings = {}
    for col in unmatched_columns:
        user_input = input(f"\nEnter the correct column name for '{col}' (or press Enter to skip): ").strip()
        if user_input:
            if user_input in column_mapping.values():
                print(f"Error: '{user_input}' is already mapped to another column. Please choose a different name.")
            else:
                column_mapping[col] = user_input
                new_mappings[col] = [user_input]

    if new_mappings:
        catalog = update_catalog(catalog, new_mappings)
        save_catalog(catalog)
        print("\nCatalog updated with new mappings.")

    return column_mapping, catalog


def etl_process(df):
    catalog = load_or_create_catalog()

    print("Original DataFrame columns:")
    print(df.columns.tolist())

    column_mapping, updated_catalog = interactive_column_mapping(df.columns, catalog)

    processed_df = process_file(df, column_mapping)

    print("\nProcessed DataFrame:")
    print(processed_df.head())
    print("\nProcessed DataFrame columns:")
    print(processed_df.columns.tolist())

    return processed_df, updated_catalog


Process file

In [ ]:
def process_client_input(file_path):
    """
    Process the client's input Excel file.

    Args:
    file_path (str): The path to the client's Excel file.

    Returns:
    tuple: A tuple containing:
        - pd.DataFrame: The processed DataFrame with mapped columns and additional required columns.
        - dict: The updated catalog.
    """
    # Load the catalog with the function
    catalog = load_or_create_catalog_ingredients()

    # Extract client name from the file name to put on table
    client_name = os.path.basename(file_path).split(' - ')[0]

    # Load the workbook
    wb = load_workbook(file_path, read_only=True, data_only=True)

    # Find the sheet with 'Data' in its name
    data_sheet = next((sheet for sheet in wb.sheetnames if 'data' in sheet.lower()), None)
    if not data_sheet:
        raise ValueError("No sheet with 'Data' in its name found in the workbook.")

    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=data_sheet)

    print("Original DataFrame columns:")
    print(df.columns.tolist())

    # Perform interactive column mapping
    column_mapping, updated_catalog = interactive_column_mapping(df.columns, catalog)

    # Process the file with the new mapping
    df = process_file(df, column_mapping)

    # Filter the DataFrame to include only the columns from our catalog
    catalog_columns = list(column_mapping.keys())
    df = df[catalog_columns]


    df['Stage'] = 'Opportunity Assessment Ingredients'
    df['Client Name'] = client_name

    print("\nProcessed DataFrame:")
    print(df.head())
    print("\nProcessed DataFrame columns:")
    print(df.columns.tolist())

    return df, updated_catalog

def run_example():
    """
    Run an example of the client input processing.

    This function processes a sample Excel file and displays the results.

    Returns:
    pd.DataFrame: The processed DataFrame.
    """
    # File path (replace with Azure Blob Storage path when moving to Azure)
    file_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Kevin's Natural Foods - Opportunity Assessment.xlsx"

    # Process the client input
    processed_df, updated_catalog = process_client_input(file_path)

    # Here you would typically save the processed DataFrame or perform further analysis
    print("\nProcessing completed. DataFrame shape:", processed_df.shape)
    print("\nFirst few rows of the processed DataFrame:")
    print(processed_df.head())

    # Display the updated catalog
    print("\nUpdated Catalog:")
    print(json.dumps(updated_catalog, indent=2))

    return processed_df

# Run the example and store the result in a global variable
df = run_example()

if __name__ == "__main__":
    pass

Original DataFrame columns:
['Itemcode', 'Item Description', 'Brand', 'Packsize', 'Vendor ID (PO)', 'Vendor Name (PO)', 'Receive Date', 'PO Number', 'PO Line #', 'Ordered Quantity', 'Order UOM', 'Quatity UOM', 'Received Quantity', 'Rec. UOM', 'Weight (LB)', 'Variance (Quantity)', 'Estimate Inventory Cost', 'Actual Inventory Cost', 'Variance (Inventory Cost)', 'Estimate Add-on Cost', 'Actual Add-on Cost', 'Variance (Add-on Cost)']
Initial column mapping:
Item ID: Itemcode
Supplier Name: Vendor Name (PO)
Item Description: Item Description
UOM: Order UOM
Quantity in UOM: Received Quantity
Total: Actual Inventory Cost
Date: Receive Date

Unmatched columns:

Processed DataFrame:
    Item ID                        Supplier Name  \
0  FKC00073  PACIFIC SPICE COMPANY INC             
1  FKC00073  PACIFIC SPICE COMPANY INC             
2  FKC00073  PACIFIC SPICE COMPANY INC             
3  FKC00073  PACIFIC SPICE COMPANY INC             
4  FKC00073  PACIFIC SPICE COMPANY INC             

    

Look Up table

In [ ]:

def generate_supplier_ID(name):
    """
    Generate a supplier ID from a supplier name.
    Example: 'AMICK FARMS INC' -> 'AFI'
    """
    # Take first letter of each word, up to 3 words
    return ''.join(word[0] for word in str(name).upper().split()[:3])

def load_ingredients_lookup(path):
    """
    Load the ingredients lookup table from Excel file.
    Returns a pandas DataFrame with supplier information.
    """
    return pd.read_excel(path)

def update_ingredients_lookup(existing_lookup, new_suppliers):
    """
    Update the ingredients lookup table with new suppliers.

    Parameters:
    existing_lookup (pd.DataFrame): Existing lookup table with 'Supplier Name' and 'Supplier ID'
    new_suppliers (list): List of new supplier names to add

    Returns:
    pd.DataFrame: Updated lookup table with new suppliers added
    """
    # Create a copy of existing lookup to avoid modifying original
    updated_lookup = existing_lookup.copy()

    # Track existing supplier IDs
    existing_ids = set(existing_lookup['Supplier ID'].dropna())

    # Process new suppliers
    new_entries = []
    for supplier in new_suppliers:
        if supplier not in existing_lookup['Supplier Name'].values:
            base_id = generate_supplier_ID(supplier)

            # If base ID already exists, append a number
            if base_id in existing_ids:
                counter = 1
                while f"{base_id}{counter:03d}" in existing_ids:
                    counter += 1
                supplier_id = f"{base_id}{counter:03d}"
            else:
                supplier_id = base_id

            existing_ids.add(supplier_id)

            new_entry = {
                'Supplier Name': supplier,
                'Supplier ID': supplier_id,
                'Item Description': '',
                'Type': '',
                'Category': '',
                'Opportunity Term': '',
                'Est. Savings % (Low)': 0.00,
                'Est. Savings % (High)': 0.00
            }
            new_entries.append(new_entry)

    # Add new entries to the lookup table
    if new_entries:
        new_entries_df = pd.DataFrame(new_entries)
        updated_lookup = pd.concat([updated_lookup, new_entries_df], ignore_index=True)

    return updated_lookup


Merge

In [ ]:

def merge_vendor_data_ingredients(client_df, ingredients_lookup):
    """
    Merge client vendor data with ingredients lookup table based on Item Description.

    Parameters:
    client_df (pd.DataFrame): Client data with columns:
        - Item ID
        - Supplier Name
        - Item Description
        - UOM
        - Quantity in UOM
        - Total
        - Date
    ingredients_lookup (pd.DataFrame): Ingredients lookup table with columns:
        - Supplier Name
        - Supplier ID
        - Item Description
        - Type
        - Category
        - Opportunity Term
        - Est. Savings % (Low)
        - Est. Savings % (High)

    Returns:
    pd.DataFrame: Merged dataframe with calculated savings
    """
    # Create copies to avoid modifying original dataframes
    df = client_df.copy()
    lookup = ingredients_lookup.copy()

    # Convert Item Description to uppercase in both dataframes for case-insensitive merge
    df['Item Description_upper'] = df['Item Description'].str.upper()
    lookup['Item Description_upper'] = lookup['Item Description'].str.upper()

    # Perform the merge
    final_df = pd.merge(
        df,
        lookup,
        on='Item Description_upper',
        how='left',
        suffixes=('', '_lookup')
    )

    # Remove the uppercase columns used for merging
    final_df = final_df.drop(['Item Description_upper'], axis=1)

    # Calculate estimated savings
    final_df['Est. Savings (Low)'] = final_df['Total'] * final_df['Est. Savings % (Low)']
    final_df['Est. Savings (High)'] = final_df['Total'] * final_df['Est. Savings % (High)']

    # Handle potential NaN values in the new columns
    final_df['Est. Savings (Low)'] = final_df['Est. Savings (Low)'].fillna(0)
    final_df['Est. Savings (High)'] = final_df['Est. Savings (High)'].fillna(0)

    # Reorder columns to a logical sequence
    columns_order = [
        'Item ID',
        'Supplier Name',
        'Supplier ID',
        'Item Description',
        'Type',
        'Category',
        'UOM',
        'Quantity in UOM',
        'Total',
        'Est. Savings % (Low)',
        'Est. Savings % (High)',
        'Est. Savings (Low)',
        'Est. Savings (High)',
        'Opportunity Term',
        'Date'
    ]

    # Only include columns that exist in the final dataframe
    final_columns = [col for col in columns_order if col in final_df.columns]
    final_df = final_df[final_columns]

    return final_df

Missing Data

In [ ]:
def create_missing_raw_data_report_ingredients(df, output_path):
    """
    Create a report of missing raw data from the client's ingredients upload.

    Parameters:
    df (pd.DataFrame): DataFrame with expected columns:
        - Item ID
        - Supplier Name
        - Item Description
        - UOM
        - Quantity in UOM
        - Total
        - Date
    output_path (str): Base path to save the Excel report (will append '_ingredients')

    Returns:
    pd.DataFrame: Records with missing data
    """
    columns_to_check = [
        'Item ID',
        'Supplier Name',
        'Item Description',
        'UOM',
        'Quantity in UOM',
        'Total',
        'Date'
    ]

    # Modify output path to include _ingredients
    base_name, ext = os.path.splitext(output_path)
    ingredients_output_path = f"{base_name}_ingredients{ext}"

    missing_mask = df[columns_to_check].isnull().any(axis=1)
    missing_data = df[missing_mask]
    missing_data.to_excel(ingredients_output_path, index=False)
    print(f"Missing raw data report saved to {ingredients_output_path}")
    return missing_data

def create_missing_vendor_info_report_ingredients(final_df, ingredients_lookup, output_path):
    """
    Create a report of missing vendor information for ingredients in the client's data.

    Parameters:
    final_df (pd.DataFrame): Merged client data
    ingredients_lookup (pd.DataFrame): Ingredients lookup table
    output_path (str): Base path to save the Excel report (will append '_ingredients')

    Returns:
    pd.DataFrame: Records with missing vendor information
    """
    columns_to_check = [
        'Supplier ID',
        'Type',
        'Category',
        'Opportunity Term',
        'Est. Savings % (Low)',
        'Est. Savings % (High)'
    ]

    # Modify output path to include _ingredients
    base_name, ext = os.path.splitext(output_path)
    ingredients_output_path = f"{base_name}_ingredients{ext}"

    client_items = final_df['Item Description'].str.upper().unique()
    vendor_info = ingredients_lookup[
        ingredients_lookup['Item Description'].str.upper().isin(client_items)
    ]

    missing_mask = vendor_info[columns_to_check].isnull().any(axis=1)
    missing_vendor_info = vendor_info[missing_mask]
    missing_vendor_info.to_excel(ingredients_output_path, index=False)
    print(f"Missing vendor information report saved to {ingredients_output_path}")
    return missing_vendor_info

def calculate_missing_data_percentage_ingredients(df):
    """
    Calculate the percentage of missing data for each column in ingredients data.

    Parameters:
    df (pd.DataFrame): DataFrame to analyze

    Returns:
    pd.Series: Percentage of missing data for each column
    """
    relevant_columns = [
        'Item ID',
        'Supplier Name',
        'Item Description',
        'UOM',
        'Quantity in UOM',
        'Total',
        'Date'
    ]

    # Only calculate for columns that exist in the DataFrame
    columns_to_check = [col for col in relevant_columns if col in df.columns]

    total_rows = len(df)
    missing_percentages = (df[columns_to_check].isnull().sum() / total_rows * 100).round(2)
    return missing_percentages.sort_values(ascending=False)

def update_df_with_missing_data_ingredients(original_df, missing_data):
    """
    Update the original ingredients DataFrame with data from the missing data file.

    Parameters:
    original_df (pd.DataFrame): Original ingredients data
    missing_data (pd.DataFrame): DataFrame containing missing data to be added

    Returns:
    pd.DataFrame: Updated DataFrame with missing data filled in
    """
    key_columns = ['Item Description', 'Supplier Name', 'Item ID']

    # Verify key columns exist
    for col in key_columns.copy():
        if col not in original_df.columns or col not in missing_data.columns:
            print(f"Warning: Key column '{col}' not found in one of the DataFrames. Skipping this column for matching.")
            key_columns.remove(col)

    if not key_columns:
        raise ValueError("No valid key columns for matching. Cannot update DataFrame.")

    # Perform the merge
    updated_df = pd.merge(
        original_df,
        missing_data,
        on=key_columns,
        how='left',
        suffixes=('', '_new')
    )

    # Update existing values with new data where available
    for col in missing_data.columns:
        if col in original_df.columns and col not in key_columns:
            new_col = f'{col}_new'
            if new_col in updated_df.columns:
                mask = updated_df[new_col].notna()
                updated_df.loc[mask, col] = updated_df.loc[mask, new_col]

    # Remove temporary columns

Process File

In [ ]:


def process_file_ingredients(df, column_mapping):
    """
    Process ingredients file by standardizing column names and data types.

    Parameters:
    df (pd.DataFrame): Raw ingredients data DataFrame
    column_mapping (dict): Mapping of standard column names to input column names
        Expected standard columns:
        - Item ID
        - Supplier Name
        - Item Description
        - UOM
        - Quantity in UOM
        - Total
        - Date

    Returns:
    pd.DataFrame: Processed DataFrame with standardized columns and data types
    """
    # Rename columns based on the mapping
    df_renamed = df.rename(columns={v: k for k, v in column_mapping.items()})

    # Check for duplicate columns
    duplicate_columns = df_renamed.columns[df_renamed.columns.duplicated()].tolist()
    if duplicate_columns:
        print(f"Warning: Duplicate columns found after renaming: {duplicate_columns}")
        # Append a suffix to duplicate columns
        for col in duplicate_columns:
            mask = df_renamed.columns == col
            df_renamed.columns = [f'{col}_ingredients_{i}' if x else col
                                for i, x in enumerate(mask)]

    # Process specific columns with appropriate data types
    if 'Date' in df_renamed.columns:
        df_renamed['Date'] = pd.to_datetime(df_renamed['Date'], errors='coerce')

    if 'Quantity in UOM' in df_renamed.columns:
        df_renamed['Quantity in UOM'] = pd.to_numeric(
            df_renamed['Quantity in UOM'],
            errors='coerce'
        )

    if 'Total' in df_renamed.columns:
        df_renamed['Total'] = pd.to_numeric(
            df_renamed['Total'],
            errors='coerce'
        )

    # Ensure string columns are properly formatted
    string_columns = ['Item ID', 'Supplier Name', 'Item Description', 'UOM']
    for col in string_columns:
        if col in df_renamed.columns:
            df_renamed[col] = df_renamed[col].astype(str).replace('nan', '')

    # Remove any leading/trailing whitespace from string columns
    for col in string_columns:
        if col in df_renamed.columns:
            df_renamed[col] = df_renamed[col].str.strip()

    # Add warning for missing required columns
    required_columns = [
        'Item ID',
        'Supplier Name',
        'Item Description',
        'UOM',
        'Quantity in UOM',
        'Total',
        'Date'
    ]

    missing_columns = [col for col in required_columns
                      if col not in df_renamed.columns]
    if missing_columns:
        print(f"Warning: Missing required columns: {missing_columns}")

    return df_renamed


Process Client input

In [ ]:
import pandas as pd
import os
from openpyxl import load_workbook

def process_client_input_ingredients(file_path):
    """
    Process the client's ingredients input Excel file.

    Args:
    file_path (str): The path to the client's Excel file.

    Returns:
    tuple: A tuple containing:
        - pd.DataFrame: The processed DataFrame
        - dict: The updated catalog
    """
    # Load the catalog
    catalog = load_or_create_catalog_ingredients()

    # Extract client name from the file name
    client_name = os.path.basename(file_path).split(' - ')[0]

    # Load the workbook
    wb = load_workbook(file_path, read_only=True, data_only=True)

    print("Available sheets in workbook:")
    print(wb.sheetnames)

    # Priority order for sheet detection
    priority_sheets = [
        'Receipt Data',  # Exact match for Receipt Data
        'Data',         # Any sheet with "Data" in the name
        'Receipt',      # Any sheet with "Receipt" in the name
        'Transaction',  # Common alternative names
        'Orders'
    ]

    data_sheet = None

    # First try exact match for Receipt Data
    if 'Receipt Data' in wb.sheetnames:
        data_sheet = 'Receipt Data'
        print("\nFound 'Receipt Data' sheet")
    else:
        # Try finding sheets containing priority keywords
        for priority in priority_sheets:
            matching_sheets = [sheet for sheet in wb.sheetnames
                             if priority.lower() in sheet.lower()]
            if matching_sheets:
                data_sheet = matching_sheets[0]
                print(f"\nFound matching sheet: {data_sheet}")
                break

    # If no automatic match, ask user to select
    if not data_sheet:
        print("\nNo data sheet found automatically. Please select from available sheets:")
        print("\nAvailable sheets:")
        for i, sheet in enumerate(wb.sheetnames, 1):
            print(f"{i}. {sheet}")

        while True:
            try:
                sheet_index = int(input("\nPlease enter the number of the sheet containing the data: ")) - 1
                if 0 <= sheet_index < len(wb.sheetnames):
                    data_sheet = wb.sheetnames[sheet_index]
                    break
                else:
                    print("Invalid sheet number. Please try again.")
            except ValueError:
                print("Please enter a valid number.")

    print(f"\nUsing sheet: {data_sheet}")

    # Preview the selected sheet before processing
    print("\nPreviewing selected sheet contents:")
    preview_df = pd.read_excel(file_path, sheet_name=data_sheet, nrows=5)
    print("\nFirst 5 rows:")
    print(preview_df.head())
    print("\nColumns found:")
    print(preview_df.columns.tolist())

    # Confirm sheet selection with user
    confirmation = input("\nIs this the correct sheet? (y/n): ").lower().strip()
    while confirmation not in ['y', 'n']:
        confirmation = input("Please enter 'y' or 'n': ").lower().strip()

    if confirmation == 'n':
        print("\nPlease run the process again and select a different sheet.")
        return None, None

    # Read the full sheet
    try:
        df = pd.read_excel(file_path, sheet_name=data_sheet)

        # Check if we need to skip rows (look for actual headers)
        if df.columns.str.contains('Unnamed:').all():
            print("\nSearching for actual data rows...")
            for skip_rows in range(10):
                temp_df = pd.read_excel(file_path, sheet_name=data_sheet, skiprows=skip_rows)
                if not temp_df.columns.str.contains('Unnamed:').all():
                    df = temp_df
                    print(f"Found headers at row {skip_rows + 1}")
                    break
    except Exception as e:
        print(f"Error reading Excel file: {str(e)}")
        raise

    print("\nProcessed DataFrame columns:")
    print(df.columns.tolist())

    # Rest of the function remains the same...
    column_mapping, updated_catalog = interactive_column_mapping_ingredients(df.columns, catalog)
    df = process_file_ingredients(df, column_mapping)

    return df, updated_catalog

def preview_sheet_contents(file_path, sheet_name):
    """
    Preview the contents of a specific sheet to help with debugging.
    """
    print(f"\nPreviewing contents of sheet '{sheet_name}':")
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, nrows=5)
        print("\nFirst 5 rows:")
        print(df.head())
        print("\nColumns:")
        print(df.columns.tolist())
        print("\nShape:", df.shape)
    except Exception as e:
        print(f"Error previewing sheet: {str(e)}")

Run Kevins

In [ ]:

def run_example_ingredients():
    """
    Run the complete ingredients processing workflow.
    """
    # Define paths
    unified_ingredients_lookup_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/unified_items_ingredients_look_up.xlsx"
    client_file_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Kevin's Natural Foods - Opportunity Assessment.xlsx"
    missing_raw_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_raw_data_ingredients.xlsx"
    missing_vendor_info_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_vendor_info_ingredients.xlsx"

    print("Starting ingredients processing workflow...")

    try:
        # Load ingredients lookup table
        print("Loading unified ingredients lookup table...")
        ingredients_lookup = load_ingredients_lookup(unified_ingredients_lookup_path)

        if ingredients_lookup is None:
            raise ValueError("load_ingredients_lookup returned None")

        print("\nIngredients lookup table columns:")
        print(ingredients_lookup.columns.tolist())

        print("\nFirst few rows of ingredients lookup table:")
        print(ingredients_lookup.head())

        # Process client input
        print("\nProcessing client input...")
        result = process_client_input_ingredients(client_file_path)

        if result is None:
            raise ValueError("process_client_input_ingredients returned None")

        if isinstance(result, tuple) and len(result) == 2:
            df, updated_catalog = result
        else:
            raise ValueError(f"Unexpected return type from process_client_input_ingredients: {type(result)}")

        print("\nClient input DataFrame columns:")
        print(df.columns.tolist())

        print("\nFirst few rows of client input DataFrame:")
        print(df.head())

        # Create missing raw data report
        print("\nCreating missing raw data report...")
        missing_raw_data = create_missing_raw_data_report_ingredients(df, missing_raw_data_path)
        print(f"Missing raw data report saved to {missing_raw_data_path}")

        # Calculate percentage of missing raw data
        raw_data_missing_percentages = calculate_missing_data_percentage_ingredients(df)
        print("\nPercentage of missing data in raw data:")
        print(raw_data_missing_percentages)

        # Get unique items from raw data
        new_items = df['Item Description'].unique().tolist()

        # Update ingredients lookup with new items
        print("\nUpdating ingredients lookup with new items...")
        ingredients_lookup = update_ingredients_lookup(ingredients_lookup, new_items)

        # Merge ingredient data
        print("\nMerging ingredient data...")
        final_df = merge_vendor_data_ingredients(df, ingredients_lookup)

        if final_df is None:
            raise ValueError("merge_vendor_data_ingredients returned None")

        # Calculate estimated savings
        if all(col in final_df.columns for col in ['Total', 'Est. Savings % (Low)', 'Est. Savings % (High)']):
            final_df['Est. Savings (Low)'] = final_df['Total'] * final_df['Est. Savings % (Low)']
            final_df['Est. Savings (High)'] = final_df['Total'] * final_df['Est. Savings % (High)']
            print("Estimated savings calculated.")
        else:
            print("Warning: Unable to calculate estimated savings. Required columns missing.")

        print("\nFinal DataFrame columns after merging:")
        print(final_df.columns.tolist())

        # Create missing vendor info report
        print("\nCreating missing ingredient info report...")
        missing_vendor_info = create_missing_vendor_info_report_ingredients(
            final_df,
            ingredients_lookup,
            missing_vendor_info_path
        )
        print(f"Missing ingredient information report saved to {missing_vendor_info_path}")

        # Calculate missing data percentages for final DataFrame
        print("\nCalculating missing data percentages...")
        final_df_missing_percentages = calculate_missing_data_percentage_ingredients(final_df)
        print("\nPercentage of missing data in final DataFrame:")
        print(final_df_missing_percentages)

        # Identify missing items
        print("\nIdentifying missing items...")
        missing_items = final_df[final_df['Supplier ID'].isnull()]['Item Description'].unique()
        print(f"Number of missing items: {len(missing_items)}")

        # Calculate and display total spend and estimated savings
        if all(col in final_df.columns for col in ['Total', 'Est. Savings (Low)', 'Est. Savings (High)']):
            total_spend = final_df['Total'].sum()
            total_low_savings = final_df['Est. Savings (Low)'].sum()
            total_high_savings = final_df['Est. Savings (High)'].sum()

            print(f"\nTotal Spend: ${total_spend:,.2f}")
            print(f"Estimated Low Savings: ${total_low_savings:,.2f} ({total_low_savings/total_spend*100:.2f}%)")
            print(f"Estimated High Savings: ${total_high_savings:,.2f} ({total_high_savings/total_spend*100:.2f}%)")

        # Save updated ingredients lookup
        print("\nSaving updated ingredients lookup...")
        ingredients_lookup.to_excel(unified_ingredients_lookup_path, index=False)
        print("Updated ingredients lookup table saved.")

        # Save updated catalog if it exists
        if updated_catalog is not None:
            print("\nSaving updated catalog...")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            updated_catalog_path = f"/content/drive/MyDrive/Seismic/Seismic Python Automation/ingredients_catalog_{timestamp}.json"
            with open(updated_catalog_path, 'w') as f:
                json.dump(updated_catalog, f, indent=4)
            print(f"Updated catalog saved to: {updated_catalog_path}")

        # Print final summary
        print("\nIngredients Processing Summary:")
        print("Raw Data Missing Percentages:")
        print(raw_data_missing_percentages)
        print("\nIngredient Info Missing Percentages:")
        vendor_info_missing_percentages = calculate_missing_data_percentage_ingredients(missing_vendor_info)
        print(vendor_info_missing_percentages)
        print("\nFinal DataFrame Missing Percentages:")
        print(final_df_missing_percentages)

        return (
            final_df,
            ingredients_lookup,
            missing_raw_data,
            missing_vendor_info,
            final_df_missing_percentages
        )

    except Exception as e:
        print(f"An error occurred in ingredients processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None

# Run the example
if __name__ == "__main__":
    results = run_example_ingredients()
    final_df, ingredients_lookup, missing_raw_data, missing_vendor_info, missing_percentages = results

    if all(v is not None for v in results):
        print("Ingredients processing completed successfully")
    else:
        print("Ingredients processing failed")

Starting ingredients processing workflow...
Loading unified ingredients lookup table...

Ingredients lookup table columns:
['Supplier Name', 'Supplier ID', 'Item Description', 'Type', 'Category', 'Opportunity Term', 'Est. Savings % (Low)', 'Est. Savings % (High)', 'Item Description_lower']

First few rows of ingredients lookup table:
                         Supplier Name Supplier ID  \
0  AMICK FARMS INC                             NaN   
1  AMICK FARMS INC                             NaN   
2  PMI PARKER MIGLIORINI INTERNATIONAL         NaN   
3  PMI PARKER MIGLIORINI INTERNATIONAL         NaN   
4  PMI PARKER MIGLIORINI INTERNATIONAL         NaN   

                      Item Description         Type  Category  \
0  NAE BNLS SKNLS CHICKEN BREAST,FRESH  Ingredients  Proteins   
1  NAE CHICKEN BRST PORTIONS, BULK      Ingredients  Proteins   
2  GAP NAE CHX FRSH BRST >.31 LBS       Ingredients  Proteins   
3  GRASS FED, CC TOP SIRLOIN MEX        Ingredients  Proteins   
4  GRASS FED, 

In [ ]:
final_df.sort_values(by='Est. Savings (Low)',ascending=False)

,Item ID,Supplier Name,Supplier ID,Item Description,Type,Category,UOM,Quantity in UOM,Total,Est. Savings % (Low),Est. Savings % (High),Est. Savings (Low),Est. Savings (High),Opportunity Term,Date
9854,RB000019,MARCUS FOOD COMPANY,NaN,"GRASS-FED, FRSH HEART OF RUMP 100VL",Ingredients,Proteins,LB,39093.280,175918.95,0.02,0.05,3518.3790,8795.9475,NaN,2024-04-25
9853,RB000019,MARCUS FOOD COMPANY,NaN,"GRASS-FED, FRSH HEART OF RUMP 100VL",Ingredients,Proteins,LB,39093.280,175918.95,0.02,0.05,3518.3790,8795.9475,NaN,2024-04-25
6111,PF000009,CRYOVAC INC,NaN,"FILM, 420MM SAUCE CRYOVAC FS5050 5M",Packaging,Flexible Packaging,RL,420.000,175527.44,0.02,0.05,3510.5488,8776.3720,NaN,2023-03-09
6112,PF000009,CRYOVAC INC,NaN,"FILM, 420MM SAUCE CRYOVAC FS5050 5M",Packaging,Flexible Packaging,RL,420.000,175527.44,0.02,0.05,3510.5488,8776.3720,NaN,2023-03-09
9820,RB000016,PMI PARKER MIGLIORINI INTERNATIONAL,NaN,"GRASS-FED, TOP RND/SRLN BEEF-FOREST",Ingredients,Proteins,LB,41844.450,173652.64,0.02,0.05,3473.0528,8682.6320,NaN,2022-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22825,WST00058,TRIBE 9 FOODS LLC,NaN,"WIP - CAULI PASTA, MACARONI 5OZ",NaN,NaN,LB,2953.125,10129.22,NaN,NaN,NaN,NaN,NaN,2023-03-29
22826,WST00058,TRIBE 9 FOODS LLC,NaN,"WIP - CAULI PASTA, MACARONI 5OZ",NaN,NaN,LB,3937.500,13505.63,NaN,NaN,NaN,NaN,NaN,2023-04-03
22827,ZOS00004,APEX DISPLAY GROUP,NaN,"POS, KNF SAUCE POUCH DISPLAY PKG",NaN,NaN,EA,1000.000,20210.00,NaN,NaN,NaN,NaN,NaN,2022-08-22
22828,ZOS00004,APEX DISPLAY GROUP,NaN,"POS, KNF SAUCE POUCH DISPLAY PKG",NaN,NaN,EA,1000.000,19000.00,NaN,NaN,NaN,NaN,NaN,2023-06-22


Modifications
skip rows where mayority headers null
go to consolidates

In [ ]:

def process_client_input_ingredients(file_path):
    """
    Process the client's ingredients input Excel file.
    Handles various sheet formats, empty rows, and header structures.

    Args:
    file_path (str): The path to the client's Excel file.

    Returns:
    tuple: A tuple containing:
        - pd.DataFrame: The processed DataFrame
        - dict: The updated catalog
    """
    try:
        # Load the catalog
        catalog = load_or_create_catalog_ingredients()

        # Extract client name from the file name
        client_name = os.path.basename(file_path).split(' - ')[0]

        # Load the workbook
        wb = load_workbook(file_path, read_only=True, data_only=True)

        print("Available sheets in workbook:")
        print(wb.sheetnames)

        # Priority order for sheet detection with row count check
        priority_sheets = [
            'Consolidated Data',  # Consolidated data sheets
            'Receipt Data',      # Receipt data sheets
            'Raw data',          # Raw data sheets
            'Data',             # Generic data sheets
            'Transaction'        # Transaction sheets
        ]

        # Gather information about all sheets
        sheet_info = []
        for sheet_name in wb.sheetnames:
            try:
                df_temp = pd.read_excel(file_path, sheet_name=sheet_name)
                row_count = len(df_temp)
                sheet_info.append({
                    'name': sheet_name,
                    'rows': row_count
                })
                print(f"Sheet '{sheet_name}' has {row_count} rows")
            except Exception as e:
                print(f"Error reading sheet '{sheet_name}': {str(e)}")

        # First try to find priority sheets with the most rows
        selected_sheet = None
        max_rows = 0

        for priority in priority_sheets:
            matching_sheets = [info for info in sheet_info
                            if priority.lower() in info['name'].lower()]
            if matching_sheets:
                best_match = max(matching_sheets, key=lambda x: x['rows'])
                if best_match['rows'] > max_rows:
                    max_rows = best_match['rows']
                    selected_sheet = best_match['name']
                    print(f"\nFound priority sheet: {selected_sheet} with {max_rows} rows")
                    break

        # If no priority match found, take the sheet with the most rows
        if not selected_sheet and sheet_info:
            best_match = max(sheet_info, key=lambda x: x['rows'])
            selected_sheet = best_match['name']
            max_rows = best_match['rows']
            print(f"\nSelected sheet with most rows: {selected_sheet} with {max_rows} rows")

        print(f"\nUsing sheet: {selected_sheet}")

        # Function to find the actual header row
        def find_header_row(df):
            """Find the row where actual headers start."""
            header_keywords = ['location', 'sku', 'product', 'description', 'type', 'code',
                           'receipt', 'transaction', 'date', 'price', 'quantity']

            for idx in range(min(len(df), 30)):  # Check first 30 rows
                row = df.iloc[idx]
                row_values = [str(val).lower().strip() for val in row.values if pd.notna(val)]

                # Skip empty rows
                if not row_values:
                    continue

                # Check if row contains common header keywords
                if any(keyword in ' '.join(row_values) for keyword in header_keywords):
                    # Verify next row contains actual data
                    if idx + 1 < len(df):
                        next_row = df.iloc[idx + 1]
                        if not next_row.isnull().all():  # Check if next row is not empty
                            return idx

                # Alternative check: Look for pattern of consistent data types
                if idx + 2 < len(df):
                    next_row = df.iloc[idx + 1]
                    next_row_values = [str(val).lower().strip() for val in next_row.values if pd.notna(val)]
                    if len(next_row_values) > 0 and any(val.isdigit() or 'receipt' in val.lower() for val in next_row_values):
                        return idx

            return 0

        # Read and process the data
        try:
            # Read first several rows to analyze structure
            preview_df = pd.read_excel(file_path, sheet_name=selected_sheet, nrows=30)
            header_row = find_header_row(preview_df)

            print(f"\nAnalyzing data structure...")
            print(f"Detected header row at index: {header_row}")

            # Read the actual data with the correct header row
            df = pd.read_excel(file_path, sheet_name=selected_sheet, skiprows=header_row)

            # Clean up column names
            df.columns = [str(col).strip() for col in df.columns]

            # Remove any completely empty rows
            df = df.dropna(how='all')

            # Remove any remaining unnamed columns that are completely empty
            unnamed_cols = [col for col in df.columns
                          if 'unnamed' in str(col).lower() and df[col].isnull().all()]
            if unnamed_cols:
                df = df.drop(columns=unnamed_cols)

            print("\nProcessed columns:")
            print(df.columns.tolist())
            print("\nFirst 5 rows of processed data:")
            print(df.head())

            # Ask for confirmation
            while True:
                confirmation = input("\nDoes this data look correct? (y/n): ").lower().strip()
                if confirmation in ['y', 'n']:
                    break
                print("Please enter 'y' or 'n'")

            if confirmation == 'n':
                # If data doesn't look correct, try alternative skiprows values
                for skip_rows in [1, 2, 3, header_row + 1]:
                    print(f"\nTrying with skip_rows = {skip_rows}")
                    df = pd.read_excel(file_path, sheet_name=selected_sheet, skiprows=skip_rows)
                    print("\nColumns:")
                    print(df.columns.tolist())
                    print("\nFirst 5 rows:")
                    print(df.head())

                    confirmation = input("\nDoes this data look correct? (y/n): ").lower().strip()
                    if confirmation == 'y':
                        break

                if confirmation == 'n':
                    print("\nUnable to find correct data structure. Please check the file format.")
                    return None, None

        except Exception as e:
            print(f"Error processing file: {str(e)}")
            return None, None

        # Process the columns
        column_mapping, updated_catalog = interactive_column_mapping_ingredients(df.columns, catalog)
        df = process_file_ingredients(df, column_mapping)

        # Add client name if not present
        if 'Client Name' not in df.columns:
            df['Client Name'] = client_name

        return df, updated_catalog

    except Exception as e:
        print(f"An error occurred while processing the file: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None


Find matching

In [ ]:
def find_matching_column(df_columns, catalog):
    """
    Find matching columns between DataFrame columns and catalog entries.
    Handles non-string values by converting to string before comparison.

    Args:
    df_columns: List or Index of DataFrame columns
    catalog: Dictionary of standard column names and their variations

    Returns:
    tuple: (column_mapping, unmatched_columns)
    """
    column_mapping = {}
    unmatched_columns = []
    df_columns = pd.Index(df_columns)

    for standard_name, variations in catalog.items():
        match_found = False
        for possible_name in variations:
            # Convert both sides to string and lowercase for comparison
            exact_match = df_columns[df_columns.astype(str).str.lower() == str(possible_name).lower()]
            if len(exact_match) > 0:
                column_mapping[standard_name] = exact_match[0]
                match_found = True
                break

            # For partial matches, also use string conversion
            partial_matches = [col for col in df_columns
                             if str(possible_name).lower() in str(col).lower()]
            if partial_matches:
                column_mapping[standard_name] = partial_matches[0]
                match_found = True
                break

        if not match_found:
            unmatched_columns.append(standard_name)

    return column_mapping, unmatched_columns

In [ ]:
def safe_str_compare(val1, val2):
    """
    Safely compare two values as strings, handling non-string types.

    Args:
    val1: First value to compare
    val2: Second value to compare

    Returns:
    bool: True if the lowercase string representations match
    """
    try:
        return str(val1).lower() == str(val2).lower()
    except:
        return False

def find_partial_matches(column, possible_name):
    """
    Find partial matches between column name and possible name, handling non-string types.

    Args:
    column: Column name to check
    possible_name: Name to look for

    Returns:
    bool: True if possible_name is found within column
    """
    try:
        return str(possible_name).lower() in str(column).lower()
    except:
        return False

def find_header_row(df):
    """
    Find the row where actual data headers start.

    Args:
    df: DataFrame to analyze

    Returns:
    int: Index of the header row
    """
    # Keywords that indicate actual data headers
    header_keywords = ['location', 'sku', 'product', 'description', 'type', 'code',
                      'receipt', 'transaction', 'date', 'price', 'quantity']

    for idx in range(min(len(df), 30)):  # Check first 30 rows
        row = df.iloc[idx]
        row_values = [str(val).lower().strip() for val in row.values if pd.notna(val)]

        # Skip empty rows
        if not row_values:
            continue

        # Check if row contains common header keywords
        if any(keyword in ' '.join(row_values) for keyword in header_keywords):
            # Verify next row contains actual data (not more headers)
            if idx + 1 < len(df):
                next_row = df.iloc[idx + 1]
                next_row_values = [str(val).lower().strip() for val in next_row.values if pd.notna(val)]
                if any(val.isdigit() for val in next_row_values):  # Check if next row has numeric values
                    return idx

        # Alternative check: Look for pattern of consistent data types in subsequent rows
        if idx + 2 < len(df):
            curr_row_types = [type(val) for val in row.values if pd.notna(val)]
            next_row_types = [type(val) for val in df.iloc[idx + 1].values if pd.notna(val)]
            if len(curr_row_types) > 0 and curr_row_types == next_row_types:
                return idx - 1  # Return previous row as header

    return 0


Run Sabrosura

In [ ]:

def run_example_ingredients():
    """
    Run the complete ingredients processing workflow.
    """
    # Define paths
    unified_ingredients_lookup_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/unified_items_ingredients_look_up.xlsx"
    client_file_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/Opp Assesments/Sabrosura - Opportunity Assessment.xlsx"
    missing_raw_data_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_raw_data_ingredients.xlsx"
    missing_vendor_info_path = "/content/drive/MyDrive/Seismic/Seismic Python Automation/missing_vendor_info_ingredients.xlsx"

    print("Starting ingredients processing workflow...")

    try:
        # Load ingredients lookup table
        print("Loading unified ingredients lookup table...")
        ingredients_lookup = load_ingredients_lookup(unified_ingredients_lookup_path)

        if ingredients_lookup is None:
            raise ValueError("load_ingredients_lookup returned None")

        print("\nIngredients lookup table columns:")
        print(ingredients_lookup.columns.tolist())

        print("\nFirst few rows of ingredients lookup table:")
        print(ingredients_lookup.head())

        # Process client input
        print("\nProcessing client input...")
        result = process_client_input_ingredients(client_file_path)

        if result is None:
            raise ValueError("process_client_input_ingredients returned None")

        if isinstance(result, tuple) and len(result) == 2:
            df, updated_catalog = result
        else:
            raise ValueError(f"Unexpected return type from process_client_input_ingredients: {type(result)}")

        print("\nClient input DataFrame columns:")
        print(df.columns.tolist())

        print("\nFirst few rows of client input DataFrame:")
        print(df.head())

        # Create missing raw data report
        print("\nCreating missing raw data report...")
        missing_raw_data = create_missing_raw_data_report_ingredients(df, missing_raw_data_path)
        print(f"Missing raw data report saved to {missing_raw_data_path}")

        # Calculate percentage of missing raw data
        raw_data_missing_percentages = calculate_missing_data_percentage_ingredients(df)
        print("\nPercentage of missing data in raw data:")
        print(raw_data_missing_percentages)

        # Get unique items from raw data
        new_items = df['Item Description'].unique().tolist()

        # Update ingredients lookup with new items
        print("\nUpdating ingredients lookup with new items...")
        ingredients_lookup = update_ingredients_lookup(ingredients_lookup, new_items)

        # Merge ingredient data
        print("\nMerging ingredient data...")
        final_df = merge_vendor_data_ingredients(df, ingredients_lookup)

        if final_df is None:
            raise ValueError("merge_vendor_data_ingredients returned None")

        # Calculate estimated savings
        if all(col in final_df.columns for col in ['Total', 'Est. Savings % (Low)', 'Est. Savings % (High)']):
            final_df['Est. Savings (Low)'] = final_df['Total'] * final_df['Est. Savings % (Low)']
            final_df['Est. Savings (High)'] = final_df['Total'] * final_df['Est. Savings % (High)']
            print("Estimated savings calculated.")
        else:
            print("Warning: Unable to calculate estimated savings. Required columns missing.")

        print("\nFinal DataFrame columns after merging:")
        print(final_df.columns.tolist())

        # Create missing vendor info report
        print("\nCreating missing ingredient info report...")
        missing_vendor_info = create_missing_vendor_info_report_ingredients(
            final_df,
            ingredients_lookup,
            missing_vendor_info_path
        )
        print(f"Missing ingredient information report saved to {missing_vendor_info_path}")

        # Calculate missing data percentages for final DataFrame
        print("\nCalculating missing data percentages...")
        final_df_missing_percentages = calculate_missing_data_percentage_ingredients(final_df)
        print("\nPercentage of missing data in final DataFrame:")
        print(final_df_missing_percentages)

        # Identify missing items
        print("\nIdentifying missing items...")
        missing_items = final_df[final_df['Supplier ID'].isnull()]['Item Description'].unique()
        print(f"Number of missing items: {len(missing_items)}")

        # Calculate and display total spend and estimated savings
        if all(col in final_df.columns for col in ['Total', 'Est. Savings (Low)', 'Est. Savings (High)']):
            total_spend = final_df['Total'].sum()
            total_low_savings = final_df['Est. Savings (Low)'].sum()
            total_high_savings = final_df['Est. Savings (High)'].sum()

            print(f"\nTotal Spend: ${total_spend:,.2f}")
            print(f"Estimated Low Savings: ${total_low_savings:,.2f} ({total_low_savings/total_spend*100:.2f}%)")
            print(f"Estimated High Savings: ${total_high_savings:,.2f} ({total_high_savings/total_spend*100:.2f}%)")

        # Save updated ingredients lookup
        print("\nSaving updated ingredients lookup...")
        ingredients_lookup.to_excel(unified_ingredients_lookup_path, index=False)
        print("Updated ingredients lookup table saved.")

        # Save updated catalog if it exists
        if updated_catalog is not None:
            print("\nSaving updated catalog...")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            updated_catalog_path = f"/content/drive/MyDrive/Seismic/Seismic Python Automation/ingredients_catalog_{timestamp}.json"
            with open(updated_catalog_path, 'w') as f:
                json.dump(updated_catalog, f, indent=4)
            print(f"Updated catalog saved to: {updated_catalog_path}")

        # Print final summary
        print("\nIngredients Processing Summary:")
        print("Raw Data Missing Percentages:")
        print(raw_data_missing_percentages)
        print("\nIngredient Info Missing Percentages:")
        vendor_info_missing_percentages = calculate_missing_data_percentage_ingredients(missing_vendor_info)
        print(vendor_info_missing_percentages)
        print("\nFinal DataFrame Missing Percentages:")
        print(final_df_missing_percentages)

        return (
            final_df,
            ingredients_lookup,
            missing_raw_data,
            missing_vendor_info,
            final_df_missing_percentages
        )

    except Exception as e:
        print(f"An error occurred in ingredients processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None

# Run the example
if __name__ == "__main__":
    results = run_example_ingredients()
    final_df, ingredients_lookup, missing_raw_data, missing_vendor_info, missing_percentages = results

    if all(v is not None for v in results):
        print("Ingredients processing completed successfully")
    else:
        print("Ingredients processing failed")

Starting ingredients processing workflow...
Loading unified ingredients lookup table...

Ingredients lookup table columns:
['Supplier Name', 'Supplier ID', 'Item Description', 'Type', 'Category', 'Opportunity Term', 'Est. Savings % (Low)', 'Est. Savings % (High)', 'Item Description_lower']

First few rows of ingredients lookup table:
                         Supplier Name Supplier ID  \
0  AMICK FARMS INC                             NaN   
1  AMICK FARMS INC                             NaN   
2  PMI PARKER MIGLIORINI INTERNATIONAL         NaN   
3  PMI PARKER MIGLIORINI INTERNATIONAL         NaN   
4  PMI PARKER MIGLIORINI INTERNATIONAL         NaN   

                      Item Description         Type  Category  \
0  NAE BNLS SKNLS CHICKEN BREAST,FRESH  Ingredients  Proteins   
1  NAE CHICKEN BRST PORTIONS, BULK      Ingredients  Proteins   
2  GAP NAE CHX FRSH BRST >.31 LBS       Ingredients  Proteins   
3  GRASS FED, CC TOP SIRLOIN MEX        Ingredients  Proteins   
4  GRASS FED, 

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Sheet ' Questionnaire' has 31 rows
Sheet 'Summary' has 51 rows
Sheet 'Consolidated Data' has 4462 rows
Sheet 'Raw data WI' has 3738 rows
Sheet 'Raw data CA' has 1459 rows
Sheet 'Seismic Categories' has 29 rows

Found priority sheet: Consolidated Data with 4462 rows

Using sheet: Consolidated Data

Analyzing data structure...
Detected header row at index: 0

Processed columns:
['Unnamed: 0', 'Unnamed: 1', '501', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', '13861301.217894038', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']

First 5 rows of processed data:
   Unnamed: 0 Unnamed: 1    501         Unnamed: 3    Unnamed: 4  \
0    Location    Product  SKU's           Txn Type  Product Code   
1  California   DP300-10      1  Receipts to Stock         DP300   
2  California   DP300-10      0  Receipts to Stock         DP300   
3  California   DP300-10      0  Receipt

In [ ]:
final_df.head()

,Item ID,Supplier Name,Supplier ID,Item Description,Type,Category,UOM,Quantity in UOM,Total,Est. Savings % (Low),Est. Savings % (High),Est. Savings (Low),Est. Savings (High),Opportunity Term,Date
0,1,SUNSHINE FPC INC,NaN,LITHO FILM RICE PUDDING 8OZ LID 24686/rl,NaN,NaN,,516488.0,4648.3920,NaN,NaN,NaN,NaN,NaN,2022-11-29
1,0,SUNSHINE FPC INC,NaN,LITHO FILM RICE PUDDING 8OZ LID 24686/rl,NaN,NaN,,3212954.0,25382.3366,NaN,NaN,NaN,NaN,NaN,2023-01-26
2,0,SUNSHINE FPC INC,NaN,LITHO FILM RICE PUDDING 8OZ LID 24686/rl,NaN,NaN,,2017302.0,16340.1462,NaN,NaN,NaN,NaN,NaN,2023-03-22
3,0,SUNSHINE FPC INC,NaN,LITHO FILM RICE PUDDING 8OZ LID 24686/rl,NaN,NaN,,1879746.0,15225.9426,NaN,NaN,NaN,NaN,NaN,2023-05-15
4,0,SUNSHINE FPC INC,NaN,LITHO FILM RICE PUDDING 8OZ LID 24686/rl,NaN,NaN,,2972970.0,23486.4630,NaN,NaN,NaN,NaN,NaN,2023-08-30


Ingredients App

In [ ]:
%%writefile ingredients_app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np




# Set page configuration
st.set_page_config(page_title="Ingredients Savings Dashboard", layout="wide")

# Custom CSS (same as before)
st.markdown("""
    <style>
    .big-font {
        font-size:30px !important;
        font-weight: bold;
    }
    .medium-font {
        font-size:20px !important;
    }
    .small-font {
        font-size:14px !important;
    }
    .metric-card {
        background-color: #f0f2f6;
        border-radius: 10px;
        padding: 15px;
        text-align: center;
    }
    </style>
    """, unsafe_allow_html=True)

@st.cache_data
def load_data():
    df = pd.read_csv('final_df.csv')
    # Print column names for debugging
    st.write("Available columns:", df.columns.tolist())

    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])

    return df

def calculate_metrics(df):
    if 'Total' not in df.columns:
        st.error("'Total' column not found in the data")
        return 0, 0, 0, 0, 0

    total_spend = df['Total'].sum()
    total_low_savings = df['Est. Savings (Low)'].sum() if 'Est. Savings (Low)' in df.columns else 0
    total_high_savings = df['Est. Savings (High)'].sum() if 'Est. Savings (High)' in df.columns else 0

    if total_spend > 0:
        avg_low_savings_pct = (total_low_savings / total_spend) * 100
        avg_high_savings_pct = (total_high_savings / total_spend) * 100
    else:
        avg_low_savings_pct = 0
        avg_high_savings_pct = 0

    return total_spend, total_low_savings, total_high_savings, avg_low_savings_pct, avg_high_savings_pct

def display_savings_table(df, title):
    st.subheader(title)

    # Create summary table grouped by Supplier Name
    summary = df.groupby('Supplier Name').agg({
        'Total': 'sum',
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).reset_index()

    # Calculate percentages
    summary['Est. Savings % (Low)'] = (summary['Est. Savings (Low)'] / summary['Total'] * 100).round(2)
    summary['Est. Savings % (High)'] = (summary['Est. Savings (High)'] / summary['Total'] * 100).round(2)

    # Add grand total
    grand_total = pd.DataFrame({
        'Supplier Name': ['Total'],
        'Total': [df['Total'].sum()],
        'Est. Savings (Low)': [df['Est. Savings (Low)'].sum()],
        'Est. Savings (High)': [df['Est. Savings (High)'].sum()],
        'Est. Savings % (Low)': [(df['Est. Savings (Low)'].sum() / df['Total'].sum() * 100).round(2)],
        'Est. Savings % (High)': [(df['Est. Savings (High)'].sum() / df['Total'].sum() * 100).round(2)]
    })

    summary = pd.concat([summary, grand_total])
    st.table(summary)

    # Create pie chart for savings distribution
    df_pie = df[df['Supplier Name'] != 'Total'].copy()
    fig = px.pie(df_pie,
                 values='Est. Savings (High)',
                 names='Supplier Name',
                 title="High Savings Distribution by Supplier")
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(height=500)
    st.plotly_chart(fig, use_container_width=True)

def display_monthly_savings(df):
    if 'Date' not in df.columns:
        st.warning("Date column not found. Cannot display monthly trends.")
        return

    st.subheader("Monthly Savings Trend")

    monthly_savings = df.groupby(df['Date'].dt.to_period('M')).agg({
        'Est. Savings (Low)': 'sum',
        'Est. Savings (High)': 'sum'
    }).reset_index()
    monthly_savings['Date'] = monthly_savings['Date'].dt.to_timestamp()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=monthly_savings['Date'],
                            y=monthly_savings['Est. Savings (Low)'],
                            mode='lines+markers',
                            name='Low Savings'))
    fig.add_trace(go.Scatter(x=monthly_savings['Date'],
                            y=monthly_savings['Est. Savings (High)'],
                            mode='lines+markers',
                            name='High Savings'))

    fig.update_layout(
        title="Monthly Savings Trend",
        xaxis_title="Month",
        yaxis_title="Savings ($)",
        height=500
    )
    st.plotly_chart(fig, use_container_width=True)

def main():
    st.markdown("<h1 style='text-align: center;'>Ingredients Savings Dashboard</h1>",
                unsafe_allow_html=True)

    # Load data
    df = load_data()

    # Sidebar filters
    st.sidebar.title("Filters")

    # Supplier filter
    suppliers = ['All'] + sorted(df['Supplier Name'].unique().tolist())
    selected_supplier = st.sidebar.selectbox("Select Supplier", suppliers)
    if selected_supplier != 'All':
        df = df[df['Supplier Name'] == selected_supplier]

    # Date filter
    if 'Date' in df.columns and not df['Date'].empty:
        min_date = df['Date'].min().date()
        max_date = df['Date'].max().date()
        date_range = st.sidebar.date_input("Select Date Range",
                                         value=[min_date, max_date],
                                         min_value=min_date,
                                         max_value=max_date)

        if len(date_range) == 2:
            df = df[(df['Date'].dt.date >= date_range[0]) &
                   (df['Date'].dt.date <= date_range[1])]

    # Calculate metrics
    metrics = calculate_metrics(df)
    total_spend, total_low_savings, total_high_savings, avg_low_pct, avg_high_pct = metrics

    # Display metrics
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.markdown('<p class="medium-font">Total Ingredient Spend</p>',
                   unsafe_allow_html=True)
        st.markdown(f'<p class="big-font">${total_spend:,.2f}</p>',
                   unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

    with col2:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.markdown('<p class="medium-font">Est. Low Savings</p>',
                   unsafe_allow_html=True)
        st.markdown(f'<p class="big-font">${total_low_savings:,.2f}</p>',
                   unsafe_allow_html=True)
        st.markdown(f'<p class="small-font">({avg_low_pct:.1f}%)</p>',
                   unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

    with col3:
        st.markdown('<div class="metric-card">', unsafe_allow_html=True)
        st.markdown('<p class="medium-font">Est. High Savings</p>',
                   unsafe_allow_html=True)
        st.markdown(f'<p class="big-font">${total_high_savings:,.2f}</p>',
                   unsafe_allow_html=True)
        st.markdown(f'<p class="small-font">({avg_high_pct:.1f}%)</p>',
                   unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

    # Visualizations
    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Top 10 Ingredients by Spend")
        top_spend = df.groupby('Item Description')['Total'].sum().nlargest(10).reset_index()
        fig = px.bar(top_spend,
                    x='Total',
                    y='Item Description',
                    orientation='h',
                    title="Top 10 Ingredients by Spend")
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

    with col2:
        st.subheader("Top 10 Suppliers by Spend")
        top_suppliers = df.groupby('Supplier Name')['Total'].sum().nlargest(10).reset_index()
        fig = px.bar(top_suppliers,
                    x='Total',
                    y='Supplier Name',
                    orientation='h',
                    title="Top 10 Suppliers by Spend")
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

    # Monthly trend
    display_monthly_savings(df)

    # Ingredients savings summary
    display_savings_table(df, "Ingredients Savings Summary")

    # Raw data view
    st.subheader("Raw Data")
    st.dataframe(df.head(1000))
    if len(df) > 1000:
        st.info(f"Showing first 1000 rows out of {len(df)} total rows.")

if __name__ == "__main__":
    main()

Writing ingredients_app.py


In [ ]:
# Save the final DataFrame
final_df.to_csv('final_df.csv', index=False)

# Import and setup ngrok
from pyngrok import ngrok
ngrok.set_auth_token("2nCrHcKsJCDT2SiVXRa56Z2LMkz_3uVedRt8Q5oT5BPb9R9Lx")

# Set up a tunnel to the full address with http protocol
public_url = ngrok.connect(addr="http://localhost:8501")  # Full address
print("Public URL:", public_url)

# Run the Streamlit app
!streamlit run ingredients_app.py &